# 1.选择文件夹

In [1]:
timing = input('am or pm: ')

am or pm: am


In [2]:
import pandas as pd
import os

In [3]:
list_file = list(os.walk('./'))[0][2]
print(list_file)

for file in list_file:
    if 'MRP' in file:
        mrp = './' + file
    elif '断货改价' in file:
        last = './' + file
    elif '改价表' in file:
        stcok = './' + file 
    elif '替代ok' in file:
        alternate = './' + file
    elif '替代update' in file:
        update = './' + file
    elif '替代料总表' in file:
        combination = './' + file
    elif '每日品类' in file:
        day_category = './' + file

print(f'MRP \t{mrp}')
print(f'last \t{last}')
print(f'stock \t{stcok}')
print(f'alternate \t{alternate}')
print(f'update \t{update}')
print(f'combination \t{combination}')
print(f'day_category \t{day_category}')

['1.替代料表整理-Bear20230724.ipynb', 'US MRP20230828-运营-总表.xlsx', '大账号-断货改价-8.29下午.xlsx', '大账号_(Attention) _3.ipynb', '改价表 8-30.xlsx', '替代ok表_20230830_095619.xlsx', '替代update表_20230830_095139.xlsx', '替代料总表08-28.xlsx', '每日品类调价 8.29.xls']
MRP 	./US MRP20230828-运营-总表.xlsx
last 	./大账号-断货改价-8.29下午.xlsx
stock 	./改价表 8-30.xlsx
alternate 	./替代ok表_20230830_095619.xlsx
update 	./替代update表_20230830_095139.xlsx
combination 	./替代料总表08-28.xlsx
day_category 	./每日品类调价 8.29.xls


# 2.替代料表

In [4]:
df_alternate = pd.read_excel(alternate).fillna('')
df_alternate['row'] = [str(i) for i in range(len(df_alternate))]
df_alternate

,主料SKU,替换料1,替换料2,替换料3,替换料4,替换料5,替换料6,row
0,USCA171876-3,USCA171876,USCA171876D,USCA171876D-3,,,,0
1,USGS37022T,USGS37022,USGS37022J,USGS37022G,,,,1
2,USGS36761T,USGS36761TB,USGS36761,,,,,2
3,USGS41570B,USGS41570,,,,,,3
4,USGS69025S,USGS69025T,USGS69025,USGS69025TB,,,,4
...,...,...,...,...,...,...,...,...
41319,USWM42837-C,USWM42837,,,,,,41319
41320,USPBB32767,USPBB32767-C,,,,,,41320
41321,USPBB32755-C,USPBB32755,,,,,,41321
41322,USPBB74436-C,USPBB74436,,,,,,41322


# 3. 计算在途总数、库存

In [5]:
# 导入改价表数据计算在途总数以及每个SKU的库存
df_stock = pd.read_excel(stcok,
                         names=['SKU', '在途总数', 'YKD库存'],
                         usecols='A,E,F'
                         ).fillna(0)
df_stock

,SKU,在途总数,YKD库存
0,USBDA04239,0.0,4
1,USBDA04868,3.0,0
2,USDLA3ZP50,0.0,3
3,USEVC271FG,8.0,0
4,USBDA04007,0.0,0
...,...,...,...
243607,USGS49820,0.0,1
243608,USWR52564C,0.0,1
243609,USCVA63523-C,0.0,2
243610,USRAD13576-C,0.0,4


In [6]:
# 计算自身以及-N以及-NC的库存
df_stock['SKU'] = [df_stock.loc[i, 'SKU'][:-2] if df_stock.loc[i, 'SKU'].endswith('-N') else df_stock.loc[i, 'SKU'][:-3] if df_stock.loc[i, 'SKU'].endswith('-NC') else df_stock.loc[i,'SKU'] for i in range(len(df_stock))]

# 汇总每个SKU的库存
df_stock = df_stock.groupby(df_stock['SKU'], as_index=False).sum()
df_stock

,SKU,在途总数,YKD库存
0,BPD8550086500+BRS31318,0.0,1
1,CAWR41163,0.0,1
2,CS20230609,0.0,0
3,CS230530,0.0,0
4,CSPJ20230615,0.0,1
...,...,...,...
73755,YHTWS58202-Y-6,0.0,0
73756,YHTWS5W1ZBA-Y-10,0.0,0
73757,YHTWS5W1ZBA-Y-8,0.0,0
73758,YHTWS802AA-Y-6,0.0,0


In [7]:
# 得到每个替代料的库存数量
df_alternate['在途总数'] = 0
df_alternate['YKD库存'] = 0

df_alternate = pd.merge(left=df_alternate, right=df_stock, how='left', left_on='主料SKU', right_on='SKU', suffixes=('','_0')).fillna(0)
df_alternate = pd.merge(left=df_alternate, right=df_stock, how='left', left_on='替换料1', right_on='SKU', suffixes=('','_1')).fillna(0)
df_alternate = pd.merge(left=df_alternate, right=df_stock, how='left', left_on='替换料2', right_on='SKU', suffixes=('','_2')).fillna(0)
df_alternate = pd.merge(left=df_alternate, right=df_stock, how='left', left_on='替换料3', right_on='SKU', suffixes=('','_3')).fillna(0)
df_alternate = pd.merge(left=df_alternate, right=df_stock, how='left', left_on='替换料4', right_on='SKU', suffixes=('','_4')).fillna(0)
df_alternate = pd.merge(left=df_alternate, right=df_stock, how='left', left_on='替换料5', right_on='SKU', suffixes=('','_5')).fillna(0)
df_alternate = pd.merge(left=df_alternate, right=df_stock, how='left', left_on='替换料6', right_on='SKU', suffixes=('','_6')).fillna(0)
df_alternate

,主料SKU,替换料1,替换料2,替换料3,替换料4,替换料5,替换料6,row,在途总数,YKD库存,...,YKD库存_3,SKU_4,在途总数_4,YKD库存_4,SKU_5,在途总数_5,YKD库存_5,SKU_6,在途总数_6,YKD库存_6
0,USCA171876-3,USCA171876,USCA171876D,USCA171876D-3,,,,0,0,0,...,12.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0
1,USGS37022T,USGS37022,USGS37022J,USGS37022G,,,,1,0,0,...,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0
2,USGS36761T,USGS36761TB,USGS36761,,,,,2,0,0,...,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0
3,USGS41570B,USGS41570,,,,,,3,0,0,...,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0
4,USGS69025S,USGS69025T,USGS69025,USGS69025TB,,,,4,0,0,...,35.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41319,USWM42837-C,USWM42837,,,,,,41319,0,0,...,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0
41320,USPBB32767,USPBB32767-C,,,,,,41320,0,0,...,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0
41321,USPBB32755-C,USPBB32755,,,,,,41321,0,0,...,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0
41322,USPBB74436-C,USPBB74436,,,,,,41322,0,0,...,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0


In [8]:
df_alternate.columns

Index(['主料SKU', '替换料1', '替换料2', '替换料3', '替换料4', '替换料5', '替换料6', 'row', '在途总数',
       'YKD库存', 'SKU', '在途总数_0', 'YKD库存_0', 'SKU_1', '在途总数_1', 'YKD库存_1',
       'SKU_2', '在途总数_2', 'YKD库存_2', 'SKU_3', '在途总数_3', 'YKD库存_3', 'SKU_4',
       '在途总数_4', 'YKD库存_4', 'SKU_5', '在途总数_5', 'YKD库存_5', 'SKU_6', '在途总数_6',
       'YKD库存_6'],
      dtype='object')

In [9]:
# 计算每个替代料的“在途总数”以及”替代料库存“
df_alternate['在途总数'] = df_alternate['在途总数_0'] + df_alternate['在途总数_1'] + df_alternate['在途总数_2'] + df_alternate['在途总数_3'] + df_alternate['在途总数_4'] + df_alternate['在途总数_5'] + df_alternate['在途总数_6'] 

df_alternate['YKD库存'] = df_alternate['YKD库存_0'] + df_alternate['YKD库存_1'] + df_alternate['YKD库存_2'] + df_alternate['YKD库存_3'] + df_alternate['YKD库存_4'] + df_alternate['YKD库存_5'] + df_alternate['YKD库存_6'] 

df_alternate = df_alternate[['主料SKU',
                           '替换料1',
                           '替换料2',
                           '替换料3',
                           '替换料4',
                           '替换料5',
                           '替换料6',
                           '在途总数',
                           'YKD库存',
                           'row']].copy()
df_alternate

,主料SKU,替换料1,替换料2,替换料3,替换料4,替换料5,替换料6,在途总数,YKD库存,row
0,USCA171876-3,USCA171876,USCA171876D,USCA171876D-3,,,,6.0,12.0,0
1,USGS37022T,USGS37022,USGS37022J,USGS37022G,,,,12.0,20.0,1
2,USGS36761T,USGS36761TB,USGS36761,,,,,4.0,3.0,2
3,USGS41570B,USGS41570,,,,,,23.0,96.0,3
4,USGS69025S,USGS69025T,USGS69025,USGS69025TB,,,,4.0,35.0,4
...,...,...,...,...,...,...,...,...,...,...
41319,USWM42837-C,USWM42837,,,,,,10.0,0.0,41319
41320,USPBB32767,USPBB32767-C,,,,,,6.0,0.0,41320
41321,USPBB32755-C,USPBB32755,,,,,,6.0,0.0,41321
41322,USPBB74436-C,USPBB74436,,,,,,15.0,0.0,41322


In [10]:
list_tmp = []

for i in df_alternate.columns[:-3]:
    df_tmp = df_alternate[[i,
                          '在途总数',
                          'YKD库存',
                          'row']]
    df_tmp.columns = ['SKU','在途总数', 'YKD库存', 'row']
    df_tmp = df_tmp[df_tmp['SKU'] != '']
    list_tmp.append(df_tmp)
    
df_position = pd.concat(list_tmp).reset_index(drop=True)
df_position

,SKU,在途总数,YKD库存,row
0,USCA171876-3,6.0,12.0,0
1,USGS37022T,12.0,20.0,1
2,USGS36761T,4.0,3.0,2
3,USGS41570B,23.0,96.0,3
4,USGS69025S,4.0,35.0,4
...,...,...,...,...
87635,USIC2A367A,0.0,1.0,36527
87636,USIC71643,0.0,4.0,36625
87637,USWR48974-C,46.0,86.0,40810
87638,USIC02680L-6,0.0,65.0,40858


In [11]:
df_stock = pd.concat([df_position,df_stock]).drop_duplicates(['SKU'], keep='first').reset_index(drop=True).fillna('')
df_stock

,SKU,在途总数,YKD库存,row
0,USCA171876-3,6.0,12.0,0
1,USGS37022T,12.0,20.0,1
2,USGS36761T,4.0,3.0,2
3,USGS41570B,23.0,96.0,3
4,USGS69025S,4.0,35.0,4
...,...,...,...,...
108933,YHTWS58202-Y-6,0.0,0.0,
108934,YHTWS5W1ZBA-Y-10,0.0,0.0,
108935,YHTWS5W1ZBA-Y-8,0.0,0.0,
108936,YHTWS802AA-Y-6,0.0,0.0,


In [12]:
df_stock_way = df_stock[['SKU', '在途总数']]
df_stock_way

,SKU,在途总数
0,USCA171876-3,6.0
1,USGS37022T,12.0
2,USGS36761T,4.0
3,USGS41570B,23.0
4,USGS69025S,4.0
...,...,...
108933,YHTWS58202-Y-6,0.0
108934,YHTWS5W1ZBA-Y-10,0.0
108935,YHTWS5W1ZBA-Y-8,0.0
108936,YHTWS802AA-Y-6,0.0


In [13]:
df_stock = df_stock[['SKU', 'YKD库存', 'row']].copy()
df_stock['总库存'] = df_stock['YKD库存']
df_stock.loc[:,'组合料库存'] = 0
df_stock = df_stock[['SKU',
                     '组合料库存',
                     'YKD库存',
                     '总库存',
                     'row']]
df_stock

,SKU,组合料库存,YKD库存,总库存,row
0,USCA171876-3,0,12.0,12.0,0
1,USGS37022T,0,20.0,20.0,1
2,USGS36761T,0,3.0,3.0,2
3,USGS41570B,0,96.0,96.0,3
4,USGS69025S,0,35.0,35.0,4
...,...,...,...,...,...
108933,YHTWS58202-Y-6,0,0.0,0.0,
108934,YHTWS5W1ZBA-Y-10,0,0.0,0.0,
108935,YHTWS5W1ZBA-Y-8,0,0.0,0.0,
108936,YHTWS802AA-Y-6,0,0.0,0.0,


# 3.2 组合料库存

In [14]:
# 从替代料总表中读取组合料关系，去除数量为0，为1，为空的情况
df_combination = pd.read_excel(combination,
                               header=1,
                               dtype={'数量':str})[['主料SKU','数量','SKU1','SKU2','SKU3','SKU4','SKU5']].fillna('')

df_combination = df_combination[(df_combination['数量'] != '0') & (df_combination['数量'] != '1') & (df_combination['数量'] != '') & (df_combination['数量'] != '未更新')].reset_index(drop=True)

df_combination

,主料SKU,数量,SKU1,SKU2,SKU3,SKU4,SKU5
0,USCA171876D-3,1+1+1,USCA51317D,USCA51318D,USCAB3376D,,
1,USWCP52105-2,1+1,USWCP02525,USWCP05210,,,
2,USGS00429LR,1+1,USGS00429-1,USGS03029-1,,,
3,USGS69015TLR,1+1,USGS69015TL-1,USGS69015TR-1,,,
4,USSKA98377LR,1+1,USSKA98376,USSKA98377,,,
...,...,...,...,...,...,...,...
4438,USBCB82021LR-C,1+1,USBCB92820-C,USBCB92821-C,,,
4439,USCAK21597LR,1+1,USCA21596,USCA21597,,,
4440,USDLA40354LR-C,1+1,USDLA40344-C,USDLA40354-C,,,
4441,USBRPS657071FR,1+1+1+1,USBPD10700,USBPD10710,USBRS54125,USBRS54126,


In [15]:
def DEduplicate(df):
    list_row = list(set(df['row'].tolist()))
    if '' in list_row:
        list_row.remove('')
    list_row.sort()
    
    dict_row = {}
    for row in list_row:
        dict_row[row] = ''
    for i in range(len(df)):
        if df.loc[i, 'row'] != '':
            dict_row[df.loc[i, 'row']] += str(i) + ';'
    
    list_duplicated = []
    for i in dict_row:
        list_row = dict_row[i].split(';')[:-1]
        list_row.sort()
        length = len(list_row)
        if length > 1:
            for j in range(length -1):
                list_duplicated.append(list_row[j])
                
    for i in list_duplicated:
        df = df.drop(int(i))
        
    return df.reset_index(drop=True),list_duplicated

In [16]:
# 左连接获得替代料关系原来的行位置，如果有重复则删除
df_combination = pd.merge(left=df_combination, right=df_position, how='left', left_on='主料SKU', right_on='SKU', suffixes=('','_0')).fillna('')
df_combination,l = DEduplicate(df_combination)

print(list(df_combination.columns))
df_combination

['主料SKU', '数量', 'SKU1', 'SKU2', 'SKU3', 'SKU4', 'SKU5', 'SKU', '在途总数', 'YKD库存', 'row']


,主料SKU,数量,SKU1,SKU2,SKU3,SKU4,SKU5,SKU,在途总数,YKD库存,row
0,USCA171876D-3,1+1+1,USCA51317D,USCA51318D,USCAB3376D,,,USCA171876D-3,6.0,12.0,0
1,USWCP52105-2,1+1,USWCP02525,USWCP05210,,,,,,,
2,USGS00429LR,1+1,USGS00429-1,USGS03029-1,,,,USGS00429LR,42.0,69.0,5
3,USGS69015TLR,1+1,USGS69015TL-1,USGS69015TR-1,,,,USGS69015TLR,5.0,13.0,17
4,USSKA98377LR,1+1,USSKA98376,USSKA98377,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...
4437,USBCB82021LR-C,1+1,USBCB92820-C,USBCB92821-C,,,,USBCB82021LR-C,10.0,0.0,40673
4438,USCAK21597LR,1+1,USCA21596,USCA21597,,,,,,,
4439,USDLA40354LR-C,1+1,USDLA40344-C,USDLA40354-C,,,,USDLA40354LR-C,10.0,0.0,40700
4440,USBRPS657071FR,1+1+1+1,USBPD10700,USBPD10710,USBRS54125,USBRS54126,,,,,


In [17]:
# 获得组合料每个SKU对应的库存
df_combination = pd.merge(left=df_combination, right=df_stock, how='left', left_on='SKU1', right_on='SKU', suffixes=('','_1')).fillna({'YKD库存_1':0})
df_combination = pd.merge(left=df_combination, right=df_stock, how='left', left_on='SKU2', right_on='SKU', suffixes=('','_2')).fillna({'YKD库存_2':0})
df_combination = pd.merge(left=df_combination, right=df_stock, how='left', left_on='SKU3', right_on='SKU', suffixes=('','_3')).fillna({'YKD库存_3':0})
df_combination = pd.merge(left=df_combination, right=df_stock, how='left', left_on='SKU4', right_on='SKU', suffixes=('','_4')).fillna({'YKD库存_4':0})
df_combination = pd.merge(left=df_combination, right=df_stock, how='left', left_on='SKU5', right_on='SKU', suffixes=('','_5')).fillna({'YKD库存_5':0})
df_combination

,主料SKU,数量,SKU1,SKU2,SKU3,SKU4,SKU5,SKU,在途总数,YKD库存,...,SKU_4,组合料库存_4,YKD库存_4,总库存_4,row_4,SKU_5,组合料库存_5,YKD库存_5,总库存_5,row_5
0,USCA171876D-3,1+1+1,USCA51317D,USCA51318D,USCAB3376D,,,USCA171876D-3,6.0,12.0,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN
1,USWCP52105-2,1+1,USWCP02525,USWCP05210,,,,,,,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN
2,USGS00429LR,1+1,USGS00429-1,USGS03029-1,,,,USGS00429LR,42.0,69.0,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN
3,USGS69015TLR,1+1,USGS69015TL-1,USGS69015TR-1,,,,USGS69015TLR,5.0,13.0,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN
4,USSKA98377LR,1+1,USSKA98376,USSKA98377,,,,,,,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4437,USBCB82021LR-C,1+1,USBCB92820-C,USBCB92821-C,,,,USBCB82021LR-C,10.0,0.0,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN
4438,USCAK21597LR,1+1,USCA21596,USCA21597,,,,,,,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN
4439,USDLA40354LR-C,1+1,USDLA40344-C,USDLA40354-C,,,,USDLA40354LR-C,10.0,0.0,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN
4440,USBRPS657071FR,1+1+1+1,USBPD10700,USBPD10710,USBRS54125,USBRS54126,,,,,...,USBRS54126,0.0,6.0,6.0,,NaN,NaN,0.0,NaN,NaN


In [18]:
df_combination.columns

Index(['主料SKU', '数量', 'SKU1', 'SKU2', 'SKU3', 'SKU4', 'SKU5', 'SKU', '在途总数',
       'YKD库存', 'row', 'SKU_1', '组合料库存', 'YKD库存_1', '总库存', 'row_1', 'SKU_2',
       '组合料库存_2', 'YKD库存_2', '总库存_2', 'row_2', 'SKU_3', '组合料库存_3', 'YKD库存_3',
       '总库存_3', 'row_3', 'SKU_4', '组合料库存_4', 'YKD库存_4', '总库存_4', 'row_4',
       'SKU_5', '组合料库存_5', 'YKD库存_5', '总库存_5', 'row_5'],
      dtype='object')

In [19]:
# 计算组合料库存，根据“数量”关系来计算
import math

df_combination['组合料库存'] = 0
for i in range(len(df_combination)):
    list_amount_number = df_combination.loc[i, '数量'].split('+')
    list_amount_stock = []
    for j in range(len(list_amount_number)):
        list_amount_stock.append(df_combination.loc[i, 'YKD库存_'+str(j+1)] / int(list_amount_number[j]))
    df_combination.loc[i, '组合料库存'] = math.floor(min(list_amount_stock))
    
print(list(df_combination.columns))
df_combination

['主料SKU', '数量', 'SKU1', 'SKU2', 'SKU3', 'SKU4', 'SKU5', 'SKU', '在途总数', 'YKD库存', 'row', 'SKU_1', '组合料库存', 'YKD库存_1', '总库存', 'row_1', 'SKU_2', '组合料库存_2', 'YKD库存_2', '总库存_2', 'row_2', 'SKU_3', '组合料库存_3', 'YKD库存_3', '总库存_3', 'row_3', 'SKU_4', '组合料库存_4', 'YKD库存_4', '总库存_4', 'row_4', 'SKU_5', '组合料库存_5', 'YKD库存_5', '总库存_5', 'row_5']


,主料SKU,数量,SKU1,SKU2,SKU3,SKU4,SKU5,SKU,在途总数,YKD库存,...,SKU_4,组合料库存_4,YKD库存_4,总库存_4,row_4,SKU_5,组合料库存_5,YKD库存_5,总库存_5,row_5
0,USCA171876D-3,1+1+1,USCA51317D,USCA51318D,USCAB3376D,,,USCA171876D-3,6.0,12.0,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN
1,USWCP52105-2,1+1,USWCP02525,USWCP05210,,,,,,,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN
2,USGS00429LR,1+1,USGS00429-1,USGS03029-1,,,,USGS00429LR,42.0,69.0,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN
3,USGS69015TLR,1+1,USGS69015TL-1,USGS69015TR-1,,,,USGS69015TLR,5.0,13.0,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN
4,USSKA98377LR,1+1,USSKA98376,USSKA98377,,,,,,,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4437,USBCB82021LR-C,1+1,USBCB92820-C,USBCB92821-C,,,,USBCB82021LR-C,10.0,0.0,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN
4438,USCAK21597LR,1+1,USCA21596,USCA21597,,,,,,,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN
4439,USDLA40354LR-C,1+1,USDLA40344-C,USDLA40354-C,,,,USDLA40354LR-C,10.0,0.0,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN
4440,USBRPS657071FR,1+1+1+1,USBPD10700,USBPD10710,USBRS54125,USBRS54126,,,,,...,USBRS54126,0.0,6.0,6.0,,NaN,NaN,0.0,NaN,NaN


In [20]:
df_combination = df_combination[['主料SKU',
                                 '组合料库存',
                                ]].copy()
df_combination = pd.merge(left=df_combination, right=df_position, how='left', left_on='主料SKU', right_on='SKU').fillna({'YKD库存':0, '在途总数':0,  'row':''})
df_combination

,主料SKU,组合料库存,SKU,在途总数,YKD库存,row
0,USCA171876D-3,0,USCA171876D-3,6.0,12.0,0
1,USWCP52105-2,0,NaN,0.0,0.0,
2,USGS00429LR,0,USGS00429LR,42.0,69.0,5
3,USGS69015TLR,0,USGS69015TLR,5.0,13.0,17
4,USSKA98377LR,0,NaN,0.0,0.0,
...,...,...,...,...,...,...
4437,USBCB82021LR-C,0,USBCB82021LR-C,10.0,0.0,40673
4438,USCAK21597LR,3,NaN,0.0,0.0,
4439,USDLA40354LR-C,0,USDLA40354LR-C,10.0,0.0,40700
4440,USBRPS657071FR,0,NaN,0.0,0.0,


In [21]:
# 根据辅助列row来区分是否有重复的替代料关系需要合并计算库存，所以要区分开row为空以及不为空的情况
df_combination_1 = df_combination[df_combination.loc[:, 'row']==''].copy()
df_combination_1['总库存'] = df_combination_1.loc[:, '组合料库存'] + df_combination_1.loc[:, 'YKD库存']
df_combination_1 = df_combination_1[['主料SKU',
                                     '组合料库存',
                                     'YKD库存',
                                     '总库存',
                                     'row']].reset_index(drop=True).copy()

df_combination_1.columns = ['SKU', '组合料库存','YKD库存','总库存', 'row']
df_combination_1

,SKU,组合料库存,YKD库存,总库存,row
0,USWCP52105-2,0,0.0,0.0,
1,USSKA98377LR,0,0.0,0.0,
2,USSKA98407LR,20,0.0,20.0,
3,USGS4549LR,0,0.0,0.0,
4,USSKA98383LR,16,0.0,16.0,
...,...,...,...,...,...
2113,USBRPS091505FR,1,0.0,1.0,
2114,USBRPR091505FR,1,0.0,1.0,
2115,USCAK21597LR,3,0.0,3.0,
2116,USBRPS657071FR,0,0.0,0.0,


In [22]:
def custom_agg(row_column):
    # 检查是否有字符串或空值，如果有，返回第一个非空字符串，否则返回空字符串
    non_empty_strings = row_column.dropna().astype(str)
    if len(non_empty_strings) > 0:
        return non_empty_strings.iloc[0]
    else:
        return ''

In [23]:
df_s = pd.concat([df_stock, df_combination_1]).reset_index(drop=True).fillna('')

df_s = df_s.groupby(df_s['SKU'], as_index=False).agg({
     '组合料库存':'sum',
     'YKD库存':'sum',
     '总库存': 'sum',             
     'row': custom_agg             
})
df_s

,SKU,组合料库存,YKD库存,总库存,row
0,BPD8550086500+BRS31318,0,1.0,1.0,
1,CAWR41163,0,1.0,1.0,
2,CS20230609,0,0.0,0.0,
3,CS230530,0,0.0,0.0,
4,CSPJ20230615,0,1.0,1.0,
...,...,...,...,...,...
108933,YHTWS58202-Y-6,0,0.0,0.0,
108934,YHTWS5W1ZBA-Y-10,0,0.0,0.0,
108935,YHTWS5W1ZBA-Y-8,0,0.0,0.0,
108936,YHTWS802AA-Y-6,0,0.0,0.0,


In [24]:
df_combination_2 = df_combination[df_combination.loc[:, 'row']!=''][['组合料库存',
                                                                    'row']]
df_combination_2.reset_index(drop=True)

,组合料库存,row
0,0,0
1,0,5
2,0,17
3,0,28
4,0,40
...,...,...
2319,0,40666
2320,0,40669
2321,0,40671
2322,0,40673


In [25]:
df_s = pd.merge(left=df_s, right=df_combination_2, how='left', left_on='row', right_on='row', suffixes=('','_x')).fillna(0)
df_s['总库存'] = df_s.loc[:, 'YKD库存'] + df_s.loc[:, '组合料库存'] + df_s.loc[:, '组合料库存_x']
df_s['组合料库存'] = df_s.loc[:, '组合料库存'] + df_s.loc[:, '组合料库存_x']
df_s

,SKU,组合料库存,YKD库存,总库存,row,组合料库存_x
0,BPD8550086500+BRS31318,0.0,1.0,1.0,,0.0
1,CAWR41163,0.0,1.0,1.0,,0.0
2,CS20230609,0.0,0.0,0.0,,0.0
3,CS230530,0.0,0.0,0.0,,0.0
4,CSPJ20230615,0.0,1.0,1.0,,0.0
...,...,...,...,...,...,...
108933,YHTWS58202-Y-6,0.0,0.0,0.0,,0.0
108934,YHTWS5W1ZBA-Y-10,0.0,0.0,0.0,,0.0
108935,YHTWS5W1ZBA-Y-8,0.0,0.0,0.0,,0.0
108936,YHTWS802AA-Y-6,0.0,0.0,0.0,,0.0


In [26]:
df_s = df_s[['SKU',
             '组合料库存',
             'YKD库存',
             '总库存']].copy()
df_stock = df_s
df_stock

,SKU,组合料库存,YKD库存,总库存
0,BPD8550086500+BRS31318,0.0,1.0,1.0
1,CAWR41163,0.0,1.0,1.0
2,CS20230609,0.0,0.0,0.0
3,CS230530,0.0,0.0,0.0
4,CSPJ20230615,0.0,1.0,1.0
...,...,...,...,...
108933,YHTWS58202-Y-6,0.0,0.0,0.0
108934,YHTWS5W1ZBA-Y-10,0.0,0.0,0.0
108935,YHTWS5W1ZBA-Y-8,0.0,0.0,0.0
108936,YHTWS802AA-Y-6,0.0,0.0,0.0


# 4.MRP表整理

In [27]:
# 导入MRP数据，整合后续的断货改价表
df_mrp = pd.read_excel(mrp,
                       header=1,
                       names=['SKU','品类','层级','身份','负责人'],
                       usecols='B,G,H,J,AR')
df_mrp

,SKU,品类,层级,身份,负责人
0,USTBC40007-C,TBC,TOP1,老品,Rosie
1,USBDA04108-2-C,BDA,TOP1,老品,Ada
2,USIFTO002-C,IF,TOP1,老品,Arya
3,USWR645FRU-C,WR,TOP10,老品,Arya
4,USOS24647H-C,OS,TOP20,老品,Rosie
...,...,...,...,...,...
56537,USWSP54521B1-4-YC,WSP,TOP20,新品,Hebe
56538,USWSP54717B1-4-YC,WSP,普通,新品,Hebe
56539,USWSP54717B1-2-YC,WSP,普通,新品,Hebe
56540,USACC11756-C,ACC,TOP10,新品,Ada


In [28]:
# 筛选除CA品类以及CA-的品类都是Hebe负责的
import pandas as pd

hebe = ['CA' if category[:3] == 'CA-' else category if category=='CA' else category for category in df_mrp.loc[:, '品类']]

hebe

['TBC',
 'BDA',
 'IF',
 'WR',
 'OS',
 'WR',
 'ET',
 'TCV',
 'BM',
 'ET',
 'FP',
 'SG',
 'SA',
 'BDA',
 'FP',
 'WS',
 'CON',
 'BM',
 'BMR',
 'IM',
 'BMR',
 'GS',
 'STW',
 'HVCP',
 'BM',
 'GS',
 'GS',
 'WS',
 'ICK',
 'ACC',
 'EAF',
 'BM',
 'GS',
 'GS',
 'WM',
 'CHC',
 'PHLS',
 'BC',
 'BM',
 'CCS',
 'GS',
 'GS',
 'HB',
 'SG',
 'WIM',
 'ABS',
 'BM',
 'DS',
 'CAF',
 'BDA',
 'BM',
 'BMR',
 'FP',
 'SG',
 'ET',
 'TBC',
 'BPS',
 'CON',
 'STR',
 'CAF',
 'BDA',
 'BM',
 'BM',
 'FP',
 'FP',
 'GS',
 'BDA',
 'AEV',
 'STR',
 'MFT',
 'OP',
 'ABS',
 'ACC',
 'ACC',
 'DLA',
 'FP',
 'SG',
 'WR',
 'WR',
 'BDA',
 'RASA',
 'TPMS',
 'FIK',
 'ACC',
 'BDA',
 'BDS',
 'BMR',
 'DLA',
 'DLA',
 'FFA',
 'FP',
 'OS',
 'OS',
 'PS',
 'THA',
 'THB',
 'WL',
 'WS',
 'THB',
 'AGSA',
 'PBB',
 'FFP',
 'PHLS',
 'PSS',
 'KS',
 'ICK',
 'STR',
 'CON',
 'FSU',
 'FI',
 'MI',
 'CAF',
 'PSR',
 'ISA',
 'ABS',
 'ABS',
 'ABS',
 'ABS',
 'BDA',
 'CA',
 'ET',
 'FP',
 'GS',
 'GS',
 'GS',
 'GS',
 'GS',
 'HB',
 'WR',
 'WR',
 'ABS',
 'TFM',
 'D

In [29]:
df_m = df_mrp
df_m['品类'] = hebe
df_m[df_m.loc[:, '品类'] == 'CA']

,SKU,品类,层级,身份,负责人
119,USCAKAU018D,CA,TOP1,老品,Hebe
198,USCAK14279-C,CA,TOP20,老品,Hebe
231,USCA22812,CA,TOP20,老品,Hebe
232,USCAKBM031,CA,TOP20,老品,Hebe
233,USCAKBM032,CA,TOP1,断货,Hebe
...,...,...,...,...,...
55631,USSBK27303-C,CA,TOP20,新品,Hebe
55632,USSBK27701-C,CA,TOP20,新品,Hebe
55633,USSBR33239-C,CA,TOP20,新品,Hebe
55888,USSBK27304-C,CA,TOP20,新品,Hebe


In [30]:
# 筛选Hebe负责的品类
df_m = df_m[(df_m.loc[:,'负责人']=='Hebe') | (df_m.loc[:, '品类']=='CA')].reset_index(drop=True)
df_m

,SKU,品类,层级,身份,负责人
0,USSA85088-1-C,SA,TOP20,断货,Hebe
1,USSTW24510-C,STW,TOP1,老品,Hebe
2,USBCB72928BLR-C,BC,TOP1,老品,Hebe
3,USHB13214-C,HB,TOP10,老品,Hebe
4,USABS5570-C,ABS,TOP20,老品,Hebe
...,...,...,...,...,...
24393,USWSP65508B1-4-YC,WSP,TOP10,新品,Hebe
24394,USWSP65508B1-2-YC,WSP,普通,新品,Hebe
24395,USWSP54521B1-4-YC,WSP,TOP20,新品,Hebe
24396,USWSP54717B1-4-YC,WSP,普通,新品,Hebe


In [31]:
# 将负责人为NaN的转为Hebe
df_m['负责人'] = df_m['负责人'].map(lambda x : 'Hebe')

df_m

,SKU,品类,层级,身份,负责人
0,USSA85088-1-C,SA,TOP20,断货,Hebe
1,USSTW24510-C,STW,TOP1,老品,Hebe
2,USBCB72928BLR-C,BC,TOP1,老品,Hebe
3,USHB13214-C,HB,TOP10,老品,Hebe
4,USABS5570-C,ABS,TOP20,老品,Hebe
...,...,...,...,...,...
24393,USWSP65508B1-4-YC,WSP,TOP10,新品,Hebe
24394,USWSP65508B1-2-YC,WSP,普通,新品,Hebe
24395,USWSP54521B1-4-YC,WSP,TOP20,新品,Hebe
24396,USWSP54717B1-4-YC,WSP,普通,新品,Hebe


In [32]:
# 与position左连接，去重复行
df_m_position = pd.merge(left=df_m, right=df_position, how='left', left_on='SKU', right_on='SKU').fillna({'YKD库存':0, 'row':''}).fillna('')

print(list(df_m_position.columns))
df_m_position

['SKU', '品类', '层级', '身份', '负责人', '在途总数', 'YKD库存', 'row']


,SKU,品类,层级,身份,负责人,在途总数,YKD库存,row
0,USSA85088-1-C,SA,TOP20,断货,Hebe,0.0,0.0,6002
1,USSTW24510-C,STW,TOP1,老品,Hebe,127.0,83.0,1497
2,USBCB72928BLR-C,BC,TOP1,老品,Hebe,146.0,73.0,16587
3,USHB13214-C,HB,TOP10,老品,Hebe,16.0,4.0,7186
4,USABS5570-C,ABS,TOP20,老品,Hebe,62.0,41.0,38443
...,...,...,...,...,...,...,...,...
24393,USWSP65508B1-4-YC,WSP,TOP10,新品,Hebe,,0.0,
24394,USWSP65508B1-2-YC,WSP,普通,新品,Hebe,,0.0,
24395,USWSP54521B1-4-YC,WSP,TOP20,新品,Hebe,,0.0,
24396,USWSP54717B1-4-YC,WSP,普通,新品,Hebe,,0.0,


In [33]:
# 获得MRP表的数据
df_m_position = pd.merge(left=df_m_position, right=df_stock, how='left', left_on='SKU', right_on='SKU', suffixes=('_position', '')).fillna(0)
df_m_position

,SKU,品类,层级,身份,负责人,在途总数,YKD库存_position,row,组合料库存,YKD库存,总库存
0,USSA85088-1-C,SA,TOP20,断货,Hebe,0.0,0.0,6002,0.0,0.0,0.0
1,USSTW24510-C,STW,TOP1,老品,Hebe,127.0,83.0,1497,0.0,83.0,83.0
2,USBCB72928BLR-C,BC,TOP1,老品,Hebe,146.0,73.0,16587,1.0,73.0,74.0
3,USHB13214-C,HB,TOP10,老品,Hebe,16.0,4.0,7186,0.0,4.0,4.0
4,USABS5570-C,ABS,TOP20,老品,Hebe,62.0,41.0,38443,0.0,41.0,41.0
...,...,...,...,...,...,...,...,...,...,...,...
24393,USWSP65508B1-4-YC,WSP,TOP10,新品,Hebe,,0.0,,0.0,0.0,0.0
24394,USWSP65508B1-2-YC,WSP,普通,新品,Hebe,,0.0,,0.0,0.0,0.0
24395,USWSP54521B1-4-YC,WSP,TOP20,新品,Hebe,,0.0,,0.0,0.0,0.0
24396,USWSP54717B1-4-YC,WSP,普通,新品,Hebe,,0.0,,0.0,0.0,0.0


In [34]:
df_mrp = df_m_position[['SKU', 
                        '品类',
                        '层级',
                        '身份',
                        'row']].copy()

df_mrp

,SKU,品类,层级,身份,row
0,USSA85088-1-C,SA,TOP20,断货,6002
1,USSTW24510-C,STW,TOP1,老品,1497
2,USBCB72928BLR-C,BC,TOP1,老品,16587
3,USHB13214-C,HB,TOP10,老品,7186
4,USABS5570-C,ABS,TOP20,老品,38443
...,...,...,...,...,...
24393,USWSP65508B1-4-YC,WSP,TOP10,新品,
24394,USWSP65508B1-2-YC,WSP,普通,新品,
24395,USWSP54521B1-4-YC,WSP,TOP20,新品,
24396,USWSP54717B1-4-YC,WSP,普通,新品,


# 5.基础整合

In [35]:
# df_last = pd.read_excel(last,
#                         header=1,
#                         usecols='C,D,U,V,X:AI,AK:AL,AO,AP,AS').fillna('')

df_last = pd.read_excel(last,
                        header=1,
                        usecols='B,C,I:V,X:Y,AB:AC').fillna('')

df_update = pd.read_excel(update,
                          header=0)

df_day_category = pd.read_excel(day_category,
                                names=['SKU', '调价记录'],
                                usecols='A,B').fillna('')

df_last['原始行号'] = [i+3 for i in range(len(df_last))]

print(df_last.columns)
df_last

Index([                  'SKU',                   '替代料',
                        '实时库存',                 '组合料库存',
                       'YKD库存',     2023-08-28 00:00:01,
           2023-08-26 00:00:01,     2023-08-25 00:00:01,
       '2023-08-28 00:00:01.1',     2023-08-27 00:00:01,
       '2023-08-26 00:00:01.1', '2023-08-25 00:00:01.1',
           2023-08-24 00:00:01,     2023-08-23 00:00:01,
           2023-08-22 00:00:01,     2023-08-21 00:00:01,
                        '改价记录',                  '断货时间',
                     '总品类调价记录',                '每日调价记录',
                        '原始行号'],
      dtype='object')


,SKU,替代料,实时库存,组合料库存,YKD库存,2023-08-28 00:00:01,2023-08-26 00:00:01,2023-08-25 00:00:01,2023-08-28 00:00:01.1,2023-08-27 00:00:01,...,2023-08-25 00:00:01.1,2023-08-24 00:00:01,2023-08-23 00:00:01,2023-08-22 00:00:01,2023-08-21 00:00:01,改价记录,断货时间,总品类调价记录,每日调价记录,原始行号
0,USBCB72928BLR-C,USBCB72928BLR,70.0,1.0,72.0,66.0,69.0,69.0,4.0,1.0,...,1.0,3.0,3.0,3.0,0.0,,,,,3
1,USPSP14045D-C,USPSP14045D,47.0,0.0,47.0,48.0,48.0,49.0,0.0,0.0,...,0.0,0.0,2.0,0.0,0.0,,,2.7-T3,,4
2,USBCB75352ZLR-C,USBCB75352ZLR,1.0,0.0,1.0,1.0,2.0,2.0,1.0,0.0,...,0.0,1.0,1.0,0.0,0.0,8.24-T10,,,,5
3,USBCB08180ZLR-C,USBCB08180ZLR,102.0,12.0,102.0,102.0,102.0,103.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,,,6.5-J2,,6
4,USPSP01008D-C,USPSP01008D,19.0,0.0,19.0,19.0,19.0,19.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,,,,,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24394,USCAK10052-C,,57.0,0.0,57.0,,,,,,...,,,,,,,,,,24397
24395,USCA01113-C,,3.0,0.0,3.0,,,,,,...,,,,,,,,,,24398
24396,USCA01114-C,,5.0,0.0,5.0,,,,,,...,,,,,,,,,,24399
24397,USCAK09028-C,,7.0,3.0,7.0,,,,,,...,,,,,,,,,,24400


In [36]:
if timing == 'am':
    df_l = df_last[['SKU', '替代料'] + list(df_last.columns[4:6+1]) + list(df_last.columns[8:14+1]) + [ '改价记录',
                                                                                                         '断货时间',
                                                                                                         '总品类调价记录',
                                                                                                         '每日调价记录',
                                                                                                         '原始行号']].copy()
    df_l.columns = list(df_l.columns[:2]) + ['库存date'] + list(df_l.columns[3:])

else:
    df_l = df_last[['SKU', '替代料'] + list(df_last.columns[5:15+1]) + ['改价记录',
                                                                         '断货时间',
                                                                         '总品类调价记录',
                                                                         '每日调价记录',
                                                                         '原始行号']].copy()

print(list(df_l.columns))
# df_l

['SKU', '替代料', '库存date', datetime.datetime(2023, 8, 28, 0, 0, 1), datetime.datetime(2023, 8, 26, 0, 0, 1), '2023-08-28 00:00:01.1', datetime.datetime(2023, 8, 27, 0, 0, 1), '2023-08-26 00:00:01.1', '2023-08-25 00:00:01.1', datetime.datetime(2023, 8, 24, 0, 0, 1), datetime.datetime(2023, 8, 23, 0, 0, 1), datetime.datetime(2023, 8, 22, 0, 0, 1), '改价记录', '断货时间', '总品类调价记录', '每日调价记录', '原始行号']


In [37]:
df_position = df_position[['SKU', 'YKD库存', 'row']]
merge_df_update = df_l
merge_df_update['note'] = ''

In [38]:
# MRP表有的SKU，断货改价表没有的，添加到断货改价表里
list_sku_note = df_mrp[~df_mrp['SKU'].isin(merge_df_update['SKU'])]['SKU'].tolist()
print(len(list_sku_note))
print(list_sku_note)

80
['USABS23730-C', 'USABS23790-C', 'USABS28190-C', 'USABS19410-C', 'USABS28210-C', 'USABS28220-C', 'USABS18890-C', 'USABS19260-C', 'USABS19310-C', 'USABS19360-C', 'USABS23330-C', 'USABS28200-C', 'USABS28230-C', 'USABS28250-C', 'USABS28330-C', 'USABS23340-C', 'USABS19320-C', 'USABS19420-C', 'USABS19530-C', 'USABS23630-C', 'USABS23750-C', 'USABS23770-C', 'USABS28240-C', 'USABS28300-C', 'USABS28310-C', 'USBCB85303B', 'USABS23360-C', 'USABS23670-C', 'USABS28290-C', 'USABS2830-C', 'USABS18840-C', 'USABS18970-C', 'USABS19160-C', 'USABS19180-C', 'USABS19200-C', 'USABS19340-C', 'USABS19450-C', 'USABS19480-C', 'USABS23230-C', 'USABS23260-C', 'USABS23280-C', 'USABS23350-C', 'USABS23560-C', 'USABS23580-C', 'USABS23640-C', 'USABS23690-C', 'USABS23710-C', 'USABS28120-C', 'USABS28140-C', 'USABS28170-C', 'USABS28260-C', 'USABS28390-C', 'USABS28440-C', 'USABS28450-C', 'USABS28480-C', 'USABS2850-C', 'USBC581802', 'USBC581902', 'USBCB85302B', 'USCAB3376KitD', 'USABS1910-C', 'USABS19210-C', 'USABS19230-

In [39]:
# MRP表有的SKU，断货改价表没有的，添加到断货改价表里
df_last_list = merge_df_update['SKU'].tolist()
length = len(merge_df_update)

for i in range(len(list_sku_note)):
    if list_sku_note[i] in df_last_list:
        merge_df_update.loc[df_last_list.index(list_sku_note[i]), 'note'] = 'last'
    else:
        merge_df_update.loc[length, 'note'] = 'mrp'
        merge_df_update.loc[length, 'SKU'] = list_sku_note[i]
        length += 1
        
merge_df_update = merge_df_update.fillna('')
merge_df_update

,SKU,替代料,库存date,2023-08-28 00:00:01,2023-08-26 00:00:01,2023-08-28 00:00:01.1,2023-08-27 00:00:01,2023-08-26 00:00:01.1,2023-08-25 00:00:01.1,2023-08-24 00:00:01,2023-08-23 00:00:01,2023-08-22 00:00:01,改价记录,断货时间,总品类调价记录,每日调价记录,原始行号,note
0,USBCB72928BLR-C,USBCB72928BLR,72.0,66.0,69.0,4.0,1.0,1.0,1.0,3.0,3.0,3.0,,,,,3.0,
1,USPSP14045D-C,USPSP14045D,47.0,48.0,48.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,,,2.7-T3,,4.0,
2,USBCB75352ZLR-C,USBCB75352ZLR,1.0,1.0,2.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,8.24-T10,,,,5.0,
3,USBCB08180ZLR-C,USBCB08180ZLR,102.0,102.0,102.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,,,6.5-J2,,6.0,
4,USPSP01008D-C,USPSP01008D,19.0,19.0,19.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,,,,,7.0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24474,USPBP93656,,,,,,,,,,,,,,,,,mrp
24475,USCA22557,,,,,,,,,,,,,,,,,mrp
24476,USCA22558,,,,,,,,,,,,,,,,,mrp
24477,USCAK22557LR,,,,,,,,,,,,,,,,,mrp


In [40]:
merge_df_update = pd.merge(left=merge_df_update, right=df_position, how='left', left_on='SKU', right_on='SKU', suffixes=('_position','')).fillna('')

print(list(merge_df_update.columns))
merge_df_update

['SKU', '替代料', '库存date', datetime.datetime(2023, 8, 28, 0, 0, 1), datetime.datetime(2023, 8, 26, 0, 0, 1), '2023-08-28 00:00:01.1', datetime.datetime(2023, 8, 27, 0, 0, 1), '2023-08-26 00:00:01.1', '2023-08-25 00:00:01.1', datetime.datetime(2023, 8, 24, 0, 0, 1), datetime.datetime(2023, 8, 23, 0, 0, 1), datetime.datetime(2023, 8, 22, 0, 0, 1), '改价记录', '断货时间', '总品类调价记录', '每日调价记录', '原始行号', 'note', 'YKD库存', 'row']


,SKU,替代料,库存date,2023-08-28 00:00:01,2023-08-26 00:00:01,2023-08-28 00:00:01.1,2023-08-27 00:00:01,2023-08-26 00:00:01.1,2023-08-25 00:00:01.1,2023-08-24 00:00:01,2023-08-23 00:00:01,2023-08-22 00:00:01,改价记录,断货时间,总品类调价记录,每日调价记录,原始行号,note,YKD库存,row
0,USBCB72928BLR-C,USBCB72928BLR,72.0,66.0,69.0,4.0,1.0,1.0,1.0,3.0,3.0,3.0,,,,,3.0,,73.0,16587
1,USPSP14045D-C,USPSP14045D,47.0,48.0,48.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,,,2.7-T3,,4.0,,47.0,15664
2,USBCB75352ZLR-C,USBCB75352ZLR,1.0,1.0,2.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,8.24-T10,,,,5.0,,1.0,16609
3,USBCB08180ZLR-C,USBCB08180ZLR,102.0,102.0,102.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,,,6.5-J2,,6.0,,102.0,16590
4,USPSP01008D-C,USPSP01008D,19.0,19.0,19.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,,,,,7.0,,19.0,16037
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24474,USPBP93656,,,,,,,,,,,,,,,,,mrp,,
24475,USCA22557,,,,,,,,,,,,,,,,,mrp,6.0,33975
24476,USCA22558,,,,,,,,,,,,,,,,,mrp,8.0,33976
24477,USCAK22557LR,,,,,,,,,,,,,,,,,mrp,7.0,30228


In [41]:
merge_df_update[merge_df_update.loc[:,'SKU']=='USBCB92640-C']
merge_df_update[merge_df_update.loc[:,'SKU']=='USBCB92640']
print(merge_df_update[merge_df_update.loc[:,'SKU']=='USCAK10052-C'])
merge_df_update[merge_df_update.loc[:,'SKU']=='USCAK10009-C']

                SKU 替代料 库存date 2023-08-28 00:00:01 2023-08-26 00:00:01  \
24394  USCAK10052-C       57.0                                           

      2023-08-28 00:00:01.1 2023-08-27 00:00:01 2023-08-26 00:00:01.1  \
24394                                                                   

      2023-08-25 00:00:01.1 2023-08-24 00:00:01 2023-08-23 00:00:01  \
24394                                                                 

      2023-08-22 00:00:01 改价记录 断货时间 总品类调价记录 每日调价记录     原始行号 note YKD库存    row  
24394                                               24397.0       57.0  32358  


,SKU,替代料,库存date,2023-08-28 00:00:01,2023-08-26 00:00:01,2023-08-28 00:00:01.1,2023-08-27 00:00:01,2023-08-26 00:00:01.1,2023-08-25 00:00:01.1,2023-08-24 00:00:01,2023-08-23 00:00:01,2023-08-22 00:00:01,改价记录,断货时间,总品类调价记录,每日调价记录,原始行号,note,YKD库存,row
24469,USCAK10009-C,,,,,,,,,,,,,,,,,mrp,57.0,32358


In [42]:
merge_df_update = pd.merge(left=merge_df_update, right=df_mrp, how='left', left_on='SKU', right_on='SKU', suffixes=('','_mrp'))

print(merge_df_update.columns)
# merge_df_update

Index([                  'SKU',                   '替代料',
                      '库存date',     2023-08-28 00:00:01,
           2023-08-26 00:00:01, '2023-08-28 00:00:01.1',
           2023-08-27 00:00:01, '2023-08-26 00:00:01.1',
       '2023-08-25 00:00:01.1',     2023-08-24 00:00:01,
           2023-08-23 00:00:01,     2023-08-22 00:00:01,
                        '改价记录',                  '断货时间',
                     '总品类调价记录',                '每日调价记录',
                        '原始行号',                  'note',
                       'YKD库存',                   'row',
                          '品类',                    '层级',
                          '身份',               'row_mrp'],
      dtype='object')


In [43]:
merge_df_update = merge_df_update.merge(df_stock_way, left_on='SKU', right_on='SKU', how='left')

print(list(merge_df_update.columns))
merge_df_update

['SKU', '替代料', '库存date', datetime.datetime(2023, 8, 28, 0, 0, 1), datetime.datetime(2023, 8, 26, 0, 0, 1), '2023-08-28 00:00:01.1', datetime.datetime(2023, 8, 27, 0, 0, 1), '2023-08-26 00:00:01.1', '2023-08-25 00:00:01.1', datetime.datetime(2023, 8, 24, 0, 0, 1), datetime.datetime(2023, 8, 23, 0, 0, 1), datetime.datetime(2023, 8, 22, 0, 0, 1), '改价记录', '断货时间', '总品类调价记录', '每日调价记录', '原始行号', 'note', 'YKD库存', 'row', '品类', '层级', '身份', 'row_mrp', '在途总数']


,SKU,替代料,库存date,2023-08-28 00:00:01,2023-08-26 00:00:01,2023-08-28 00:00:01.1,2023-08-27 00:00:01,2023-08-26 00:00:01.1,2023-08-25 00:00:01.1,2023-08-24 00:00:01,...,每日调价记录,原始行号,note,YKD库存,row,品类,层级,身份,row_mrp,在途总数
0,USBCB72928BLR-C,USBCB72928BLR,72.0,66.0,69.0,4.0,1.0,1.0,1.0,3.0,...,,3.0,,73.0,16587,BC,TOP1,老品,16587,146.0
1,USPSP14045D-C,USPSP14045D,47.0,48.0,48.0,0.0,0.0,0.0,0.0,0.0,...,,4.0,,47.0,15664,PSP,TOP20,老品,15664,6.0
2,USBCB75352ZLR-C,USBCB75352ZLR,1.0,1.0,2.0,1.0,0.0,0.0,0.0,1.0,...,,5.0,,1.0,16609,BC,TOP20,断货,16609,34.0
3,USBCB08180ZLR-C,USBCB08180ZLR,102.0,102.0,102.0,0.0,0.0,0.0,1.0,1.0,...,,6.0,,102.0,16590,BC,TOP1,老品,16590,15.0
4,USPSP01008D-C,USPSP01008D,19.0,19.0,19.0,0.0,0.0,0.0,0.0,0.0,...,,7.0,,19.0,16037,PSP,普通,老品,16037,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24474,USPBP93656,,,,,,,,,,...,,,mrp,,,PBP,普通,断货,,6.0
24475,USCA22557,,,,,,,,,,...,,,mrp,6.0,33975,CA,普通,新品,33975,12.0
24476,USCA22558,,,,,,,,,,...,,,mrp,8.0,33976,CA,普通,新品,33976,13.0
24477,USCAK22557LR,,,,,,,,,,...,,,mrp,7.0,30228,CA,TOP20,新品,30228,4.0


In [44]:
merge_df_update = pd.merge(left=merge_df_update, right=df_stock, how='left', left_on='SKU',right_on='SKU', suffixes=('', '_stock'))
merge_df_update['YKD库存'] = merge_df_update['YKD库存_stock']
print(merge_df_update.columns)
merge_df_update

Index([                  'SKU',                   '替代料',
                      '库存date',     2023-08-28 00:00:01,
           2023-08-26 00:00:01, '2023-08-28 00:00:01.1',
           2023-08-27 00:00:01, '2023-08-26 00:00:01.1',
       '2023-08-25 00:00:01.1',     2023-08-24 00:00:01,
           2023-08-23 00:00:01,     2023-08-22 00:00:01,
                        '改价记录',                  '断货时间',
                     '总品类调价记录',                '每日调价记录',
                        '原始行号',                  'note',
                       'YKD库存',                   'row',
                          '品类',                    '层级',
                          '身份',               'row_mrp',
                        '在途总数',                 '组合料库存',
                 'YKD库存_stock',                   '总库存'],
      dtype='object')


,SKU,替代料,库存date,2023-08-28 00:00:01,2023-08-26 00:00:01,2023-08-28 00:00:01.1,2023-08-27 00:00:01,2023-08-26 00:00:01.1,2023-08-25 00:00:01.1,2023-08-24 00:00:01,...,YKD库存,row,品类,层级,身份,row_mrp,在途总数,组合料库存,YKD库存_stock,总库存
0,USBCB72928BLR-C,USBCB72928BLR,72.0,66.0,69.0,4.0,1.0,1.0,1.0,3.0,...,73.0,16587,BC,TOP1,老品,16587,146.0,1.0,73.0,74.0
1,USPSP14045D-C,USPSP14045D,47.0,48.0,48.0,0.0,0.0,0.0,0.0,0.0,...,47.0,15664,PSP,TOP20,老品,15664,6.0,0.0,47.0,47.0
2,USBCB75352ZLR-C,USBCB75352ZLR,1.0,1.0,2.0,1.0,0.0,0.0,0.0,1.0,...,1.0,16609,BC,TOP20,断货,16609,34.0,0.0,1.0,1.0
3,USBCB08180ZLR-C,USBCB08180ZLR,102.0,102.0,102.0,0.0,0.0,0.0,1.0,1.0,...,102.0,16590,BC,TOP1,老品,16590,15.0,12.0,102.0,114.0
4,USPSP01008D-C,USPSP01008D,19.0,19.0,19.0,0.0,0.0,0.0,0.0,0.0,...,19.0,16037,PSP,普通,老品,16037,0.0,0.0,19.0,19.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24474,USPBP93656,,,,,,,,,,...,0.0,,PBP,普通,断货,,6.0,0.0,0.0,0.0
24475,USCA22557,,,,,,,,,,...,6.0,33975,CA,普通,新品,33975,12.0,0.0,6.0,6.0
24476,USCA22558,,,,,,,,,,...,8.0,33976,CA,普通,新品,33976,13.0,0.0,8.0,8.0
24477,USCAK22557LR,,,,,,,,,,...,7.0,30228,CA,TOP20,新品,30228,4.0,6.0,7.0,13.0


In [45]:
# 与替代料匹配更新SKU
merge_df = merge_df_update.merge(df_update, left_on='SKU', right_on='替代料', how='left', suffixes=('','_x')).fillna('')

# 更新SKU
if '替代料_x' in merge_df.columns:
    merge_df['SKU'] = merge_df.apply(lambda row: row['SKU'] if row['替代料_x']=='' else row['主料SKU'], axis=1)
    
    merge_df.drop(columns=['替代料_x', '主料SKU'], inplace=True)
    
print(list(merge_df.columns))
merge_df

['SKU', '替代料', '库存date', datetime.datetime(2023, 8, 28, 0, 0, 1), datetime.datetime(2023, 8, 26, 0, 0, 1), '2023-08-28 00:00:01.1', datetime.datetime(2023, 8, 27, 0, 0, 1), '2023-08-26 00:00:01.1', '2023-08-25 00:00:01.1', datetime.datetime(2023, 8, 24, 0, 0, 1), datetime.datetime(2023, 8, 23, 0, 0, 1), datetime.datetime(2023, 8, 22, 0, 0, 1), '改价记录', '断货时间', '总品类调价记录', '每日调价记录', '原始行号', 'note', 'YKD库存', 'row', '品类', '层级', '身份', 'row_mrp', '在途总数', '组合料库存', 'YKD库存_stock', '总库存']


,SKU,替代料,库存date,2023-08-28 00:00:01,2023-08-26 00:00:01,2023-08-28 00:00:01.1,2023-08-27 00:00:01,2023-08-26 00:00:01.1,2023-08-25 00:00:01.1,2023-08-24 00:00:01,...,YKD库存,row,品类,层级,身份,row_mrp,在途总数,组合料库存,YKD库存_stock,总库存
0,USBCB72928BLR-C,USBCB72928BLR,72.0,66.0,69.0,4.0,1.0,1.0,1.0,3.0,...,73.0,16587,BC,TOP1,老品,16587,146.0,1.0,73.0,74.0
1,USPSP14045D-C,USPSP14045D,47.0,48.0,48.0,0.0,0.0,0.0,0.0,0.0,...,47.0,15664,PSP,TOP20,老品,15664,6.0,0.0,47.0,47.0
2,USBCB75352ZLR-C,USBCB75352ZLR,1.0,1.0,2.0,1.0,0.0,0.0,0.0,1.0,...,1.0,16609,BC,TOP20,断货,16609,34.0,0.0,1.0,1.0
3,USBCB08180ZLR-C,USBCB08180ZLR,102.0,102.0,102.0,0.0,0.0,0.0,1.0,1.0,...,102.0,16590,BC,TOP1,老品,16590,15.0,12.0,102.0,114.0
4,USPSP01008D-C,USPSP01008D,19.0,19.0,19.0,0.0,0.0,0.0,0.0,0.0,...,19.0,16037,PSP,普通,老品,16037,0.0,0.0,19.0,19.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24474,USPBP93656,,,,,,,,,,...,0.0,,PBP,普通,断货,,6.0,0.0,0.0,0.0
24475,USCA01113-C,,,,,,,,,,...,6.0,33975,CA,普通,新品,33975,12.0,0.0,6.0,6.0
24476,USCA01114-C,,,,,,,,,,...,8.0,33976,CA,普通,新品,33976,13.0,0.0,8.0,8.0
24477,USCAK09028-C,,,,,,,,,,...,7.0,30228,CA,TOP20,新品,30228,4.0,6.0,7.0,13.0


In [46]:
merge_df,l = DEduplicate(merge_df)

In [47]:
merge_df[merge_df.loc[:,'SKU']=='USCAK10052-C']['note']
# merge_df[merge_df.loc[:,'SKU']=='USCAK10009-C']

24393    mrp
Name: note, dtype: object

In [48]:
df = merge_df
df['改价'] = ''
df['weekly_sale'] = ''
df['改价+断货'] = ''
df['总品类调价记录_2'] = ''
df['合计改价记录'] = ''

if timing == 'am':
    df = df[['层级',
             'SKU',
             '替代料',
             '品类',
             '身份',
             '在途总数',
             '改价',
             '组合料库存',
             'YKD库存'] + list(df.columns)[2:11+1] + ['weekly_sale',
                                                       '改价记录', 
                                                       '断货时间', 
                                                       '改价+断货',
                                                       '总库存',
                                                       '总品类调价记录',
                                                       '每日调价记录',
                                                       '总品类调价记录_2',
                                                       '合计改价记录',
                                                       'note',
                                                       '原始行号',
                                                       'row']].copy()
else:
    df = df[['层级',
             'SKU',
             '替代料',
             '品类',
             '身份',
             '在途总数',
             '改价',
             '组合料库存',
             'YKD库存'] + list(df.columns)[2:12+1] + ['weekly_sale',
                                                       '改价记录', 
                                                       '断货时间', 
                                                       '改价+断货',
                                                       '总库存',
                                                       '总品类调价记录',
                                                       '每日调价记录',
                                                       '总品类调价记录_2',
                                                       '合计改价记录',
                                                       'note',
                                                       '原始行号',
                                                       'row']].copy()
print(list(df.columns))
df

['层级', 'SKU', '替代料', '品类', '身份', '在途总数', '改价', '组合料库存', 'YKD库存', '库存date', datetime.datetime(2023, 8, 28, 0, 0, 1), datetime.datetime(2023, 8, 26, 0, 0, 1), '2023-08-28 00:00:01.1', datetime.datetime(2023, 8, 27, 0, 0, 1), '2023-08-26 00:00:01.1', '2023-08-25 00:00:01.1', datetime.datetime(2023, 8, 24, 0, 0, 1), datetime.datetime(2023, 8, 23, 0, 0, 1), datetime.datetime(2023, 8, 22, 0, 0, 1), 'weekly_sale', '改价记录', '断货时间', '改价+断货', '总库存', '总品类调价记录', '每日调价记录', '总品类调价记录_2', '合计改价记录', 'note', '原始行号', 'row']


,层级,SKU,替代料,品类,身份,在途总数,改价,组合料库存,YKD库存,库存date,...,断货时间,改价+断货,总库存,总品类调价记录,每日调价记录,总品类调价记录_2,合计改价记录,note,原始行号,row
0,TOP1,USBCB72928BLR-C,USBCB72928BLR,BC,老品,146.0,,1.0,73.0,72.0,...,,,74.0,,,,,,3.0,16587
1,TOP20,USPSP14045D-C,USPSP14045D,PSP,老品,6.0,,0.0,47.0,47.0,...,,,47.0,2.7-T3,,,,,4.0,15664
2,TOP20,USBCB75352ZLR-C,USBCB75352ZLR,BC,断货,34.0,,0.0,1.0,1.0,...,,,1.0,,,,,,5.0,16609
3,TOP1,USBCB08180ZLR-C,USBCB08180ZLR,BC,老品,15.0,,12.0,102.0,102.0,...,,,114.0,6.5-J2,,,,,6.0,16590
4,普通,USPSP01008D-C,USPSP01008D,PSP,老品,0.0,,0.0,19.0,19.0,...,,,19.0,,,,,,7.0,16037
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24398,普通,USPBP93656,,PBP,断货,6.0,,0.0,0.0,,...,,,0.0,,,,,mrp,,
24399,普通,USCA01113-C,,CA,新品,12.0,,0.0,6.0,,...,,,6.0,,,,,mrp,,33975
24400,普通,USCA01114-C,,CA,新品,13.0,,0.0,8.0,,...,,,8.0,,,,,mrp,,33976
24401,TOP20,USCAK09028-C,,CA,新品,4.0,,6.0,7.0,,...,,,13.0,,,,,mrp,,30228


# 6.其他信息

In [49]:
df_missing = pd.read_excel(stcok,
                           sheet_name=1,
                           names=['SKU', '无Record ID'],
                           usecols='D,F')

df_missing = df_missing.groupby('SKU', as_index=False).sum()

df_missing

,SKU,无Record ID
0,USABS00417,1
1,USABS10220-C,1
2,USABS11330-C,2
3,USABS12740-C,2
4,USABS1450-C,1
...,...,...
2106,USWS8A068U-C,1
2107,USWS95545,1
2108,USWSL58810,4
2109,USWSP54530B1-4-YC,1


In [50]:
df = pd.merge(left=df, right=df_missing, how='left', left_on='SKU', right_on='SKU').fillna(0)

print(list(df.columns))
df

['层级', 'SKU', '替代料', '品类', '身份', '在途总数', '改价', '组合料库存', 'YKD库存', '库存date', datetime.datetime(2023, 8, 28, 0, 0, 1), datetime.datetime(2023, 8, 26, 0, 0, 1), '2023-08-28 00:00:01.1', datetime.datetime(2023, 8, 27, 0, 0, 1), '2023-08-26 00:00:01.1', '2023-08-25 00:00:01.1', datetime.datetime(2023, 8, 24, 0, 0, 1), datetime.datetime(2023, 8, 23, 0, 0, 1), datetime.datetime(2023, 8, 22, 0, 0, 1), 'weekly_sale', '改价记录', '断货时间', '改价+断货', '总库存', '总品类调价记录', '每日调价记录', '总品类调价记录_2', '合计改价记录', 'note', '原始行号', 'row', '无Record ID']


,层级,SKU,替代料,品类,身份,在途总数,改价,组合料库存,YKD库存,库存date,...,改价+断货,总库存,总品类调价记录,每日调价记录,总品类调价记录_2,合计改价记录,note,原始行号,row,无Record ID
0,TOP1,USBCB72928BLR-C,USBCB72928BLR,BC,老品,146.0,,1.0,73.0,72.0,...,,74.0,,,,,,3.0,16587,3.0
1,TOP20,USPSP14045D-C,USPSP14045D,PSP,老品,6.0,,0.0,47.0,47.0,...,,47.0,2.7-T3,,,,,4.0,15664,0.0
2,TOP20,USBCB75352ZLR-C,USBCB75352ZLR,BC,断货,34.0,,0.0,1.0,1.0,...,,1.0,,,,,,5.0,16609,0.0
3,TOP1,USBCB08180ZLR-C,USBCB08180ZLR,BC,老品,15.0,,12.0,102.0,102.0,...,,114.0,6.5-J2,,,,,6.0,16590,0.0
4,普通,USPSP01008D-C,USPSP01008D,PSP,老品,0.0,,0.0,19.0,19.0,...,,19.0,,,,,,7.0,16037,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24398,普通,USPBP93656,,PBP,断货,6.0,,0.0,0.0,,...,,0.0,,,,,mrp,,,0.0
24399,普通,USCA01113-C,,CA,新品,12.0,,0.0,6.0,,...,,6.0,,,,,mrp,,33975,0.0
24400,普通,USCA01114-C,,CA,新品,13.0,,0.0,8.0,,...,,8.0,,,,,mrp,,33976,0.0
24401,TOP20,USCAK09028-C,,CA,新品,4.0,,6.0,7.0,,...,,13.0,,,,,mrp,,30228,0.0


In [51]:
df_missing = pd.merge(left=df_missing, right=df_position, how='left', left_on='SKU', right_on='SKU').fillna('')
df_missing = df_missing[df_missing.loc[:,'row'] != ''][['SKU', '无Record ID']].copy()
df_missing = df_missing.groupby('SKU', as_index=False).sum()
df_missing

,SKU,无Record ID
0,USABS10220-C,1
1,USABS11330-C,2
2,USABS12740-C,2
3,USABS1450-C,1
4,USABS1460-C,1
...,...,...
1559,USWS46826-C,1
1560,USWS48893-C,1
1561,USWS6A663U-C,1
1562,USWS802AA-C,1


In [52]:
df = pd.merge(left=df, right=df_missing, how='left', left_on='SKU', right_on='SKU', suffixes=('_missing1', '_missing2')).fillna('')

print(df.columns)
df

Index([                   '层级',                   'SKU',
                         '替代料',                    '品类',
                          '身份',                  '在途总数',
                          '改价',                 '组合料库存',
                       'YKD库存',                '库存date',
           2023-08-28 00:00:01,     2023-08-26 00:00:01,
       '2023-08-28 00:00:01.1',     2023-08-27 00:00:01,
       '2023-08-26 00:00:01.1', '2023-08-25 00:00:01.1',
           2023-08-24 00:00:01,     2023-08-23 00:00:01,
           2023-08-22 00:00:01,           'weekly_sale',
                        '改价记录',                  '断货时间',
                       '改价+断货',                   '总库存',
                     '总品类调价记录',                '每日调价记录',
                   '总品类调价记录_2',                '合计改价记录',
                        'note',                  '原始行号',
                         'row',   '无Record ID_missing1',
         '无Record ID_missing2'],
      dtype='object')


,层级,SKU,替代料,品类,身份,在途总数,改价,组合料库存,YKD库存,库存date,...,总库存,总品类调价记录,每日调价记录,总品类调价记录_2,合计改价记录,note,原始行号,row,无Record ID_missing1,无Record ID_missing2
0,TOP1,USBCB72928BLR-C,USBCB72928BLR,BC,老品,146.0,,1.0,73.0,72.0,...,74.0,,,,,,3.0,16587,3.0,3.0
1,TOP20,USPSP14045D-C,USPSP14045D,PSP,老品,6.0,,0.0,47.0,47.0,...,47.0,2.7-T3,,,,,4.0,15664,0.0,
2,TOP20,USBCB75352ZLR-C,USBCB75352ZLR,BC,断货,34.0,,0.0,1.0,1.0,...,1.0,,,,,,5.0,16609,0.0,
3,TOP1,USBCB08180ZLR-C,USBCB08180ZLR,BC,老品,15.0,,12.0,102.0,102.0,...,114.0,6.5-J2,,,,,6.0,16590,0.0,
4,普通,USPSP01008D-C,USPSP01008D,PSP,老品,0.0,,0.0,19.0,19.0,...,19.0,,,,,,7.0,16037,0.0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24398,普通,USPBP93656,,PBP,断货,6.0,,0.0,0.0,,...,0.0,,,,,mrp,,,0.0,
24399,普通,USCA01113-C,,CA,新品,12.0,,0.0,6.0,,...,6.0,,,,,mrp,,33975,0.0,
24400,普通,USCA01114-C,,CA,新品,13.0,,0.0,8.0,,...,8.0,,,,,mrp,,33976,0.0,
24401,TOP20,USCAK09028-C,,CA,新品,4.0,,6.0,7.0,,...,13.0,,,,,mrp,,30228,0.0,


In [53]:
df.columns

Index([                   '层级',                   'SKU',
                         '替代料',                    '品类',
                          '身份',                  '在途总数',
                          '改价',                 '组合料库存',
                       'YKD库存',                '库存date',
           2023-08-28 00:00:01,     2023-08-26 00:00:01,
       '2023-08-28 00:00:01.1',     2023-08-27 00:00:01,
       '2023-08-26 00:00:01.1', '2023-08-25 00:00:01.1',
           2023-08-24 00:00:01,     2023-08-23 00:00:01,
           2023-08-22 00:00:01,           'weekly_sale',
                        '改价记录',                  '断货时间',
                       '改价+断货',                   '总库存',
                     '总品类调价记录',                '每日调价记录',
                   '总品类调价记录_2',                '合计改价记录',
                        'note',                  '原始行号',
                         'row',   '无Record ID_missing1',
         '无Record ID_missing2'],
      dtype='object')

In [54]:
df['无Record ID'] = [df.loc[i,'无Record ID_missing2'] if df.loc[i,'无Record ID_missing2'] != '' else df.loc[i,'无Record ID_missing1'] for i in range(len(df))]
df['无Record ID'] = pd.to_numeric(df['无Record ID'], errors='coerce')
df[ 'YKD库存'] = pd.to_numeric(df['YKD库存'], errors='coerce')
df['总库存'] = pd.to_numeric(df['总库存'], errors='coerce')
df['实时库存'] = [df.loc[i, 'YKD库存'] - df.loc[i, '无Record ID'] if df.loc[i, 'note'] != 'last' else '' for i in range(len(df))]
df.loc[:,'总库存'] = df.loc[:, '总库存'] - df.loc[:, '无Record ID']
print(list(df.columns))
df

['层级', 'SKU', '替代料', '品类', '身份', '在途总数', '改价', '组合料库存', 'YKD库存', '库存date', datetime.datetime(2023, 8, 28, 0, 0, 1), datetime.datetime(2023, 8, 26, 0, 0, 1), '2023-08-28 00:00:01.1', datetime.datetime(2023, 8, 27, 0, 0, 1), '2023-08-26 00:00:01.1', '2023-08-25 00:00:01.1', datetime.datetime(2023, 8, 24, 0, 0, 1), datetime.datetime(2023, 8, 23, 0, 0, 1), datetime.datetime(2023, 8, 22, 0, 0, 1), 'weekly_sale', '改价记录', '断货时间', '改价+断货', '总库存', '总品类调价记录', '每日调价记录', '总品类调价记录_2', '合计改价记录', 'note', '原始行号', 'row', '无Record ID_missing1', '无Record ID_missing2', '无Record ID', '实时库存']


,层级,SKU,替代料,品类,身份,在途总数,改价,组合料库存,YKD库存,库存date,...,每日调价记录,总品类调价记录_2,合计改价记录,note,原始行号,row,无Record ID_missing1,无Record ID_missing2,无Record ID,实时库存
0,TOP1,USBCB72928BLR-C,USBCB72928BLR,BC,老品,146.0,,1.0,73.0,72.0,...,,,,,3.0,16587,3.0,3.0,3.0,70.0
1,TOP20,USPSP14045D-C,USPSP14045D,PSP,老品,6.0,,0.0,47.0,47.0,...,,,,,4.0,15664,0.0,,0.0,47.0
2,TOP20,USBCB75352ZLR-C,USBCB75352ZLR,BC,断货,34.0,,0.0,1.0,1.0,...,,,,,5.0,16609,0.0,,0.0,1.0
3,TOP1,USBCB08180ZLR-C,USBCB08180ZLR,BC,老品,15.0,,12.0,102.0,102.0,...,,,,,6.0,16590,0.0,,0.0,102.0
4,普通,USPSP01008D-C,USPSP01008D,PSP,老品,0.0,,0.0,19.0,19.0,...,,,,,7.0,16037,0.0,,0.0,19.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24398,普通,USPBP93656,,PBP,断货,6.0,,0.0,0.0,,...,,,,mrp,,,0.0,,0.0,0.0
24399,普通,USCA01113-C,,CA,新品,12.0,,0.0,6.0,,...,,,,mrp,,33975,0.0,,0.0,6.0
24400,普通,USCA01114-C,,CA,新品,13.0,,0.0,8.0,,...,,,,mrp,,33976,0.0,,0.0,8.0
24401,TOP20,USCAK09028-C,,CA,新品,4.0,,6.0,7.0,,...,,,,mrp,,30228,0.0,,0.0,7.0


In [55]:
df_tmp = df
if timing == 'am':
    df_tmp = df_tmp[['层级',
             'SKU',
             '替代料',
             '品类',
             '身份',
             '在途总数',
             '改价',
             '无Record ID',
             '实时库存',
             '组合料库存',
             'YKD库存'] + list(df_tmp.columns)[9:18+1] + ['weekly_sale',
                                                       '改价记录', 
                                                       '断货时间', 
                                                       '改价+断货',
                                                       '总库存',
                                                       '总品类调价记录',
                                                       '每日调价记录',
                                                       '总品类调价记录_2',
                                                       '合计改价记录',
                                                       'note',
                                                       '原始行号',
                                                       'row']].copy()
else:
    df_tmp = df_tmp[['层级',
             'SKU',
             '替代料',
             '品类',
             '身份',
             '在途总数',
             '改价',
             '无Record ID',
             '实时库存',
             '组合料库存',
             'YKD库存'] + list(df_tmp.columns)[9:19+1] + ['weekly_sale',
                                                       '改价记录', 
                                                       '断货时间', 
                                                       '改价+断货',
                                                       '总库存',
                                                       '总品类调价记录',
                                                       '每日调价记录',
                                                       '总品类调价记录_2',
                                                       '合计改价记录',
                                                       'note',
                                                       '原始行号',
                                                       'row']].copy()
print(list(df_tmp.columns))
df = df_tmp
df

['层级', 'SKU', '替代料', '品类', '身份', '在途总数', '改价', '无Record ID', '实时库存', '组合料库存', 'YKD库存', '库存date', datetime.datetime(2023, 8, 28, 0, 0, 1), datetime.datetime(2023, 8, 26, 0, 0, 1), '2023-08-28 00:00:01.1', datetime.datetime(2023, 8, 27, 0, 0, 1), '2023-08-26 00:00:01.1', '2023-08-25 00:00:01.1', datetime.datetime(2023, 8, 24, 0, 0, 1), datetime.datetime(2023, 8, 23, 0, 0, 1), datetime.datetime(2023, 8, 22, 0, 0, 1), 'weekly_sale', '改价记录', '断货时间', '改价+断货', '总库存', '总品类调价记录', '每日调价记录', '总品类调价记录_2', '合计改价记录', 'note', '原始行号', 'row']


,层级,SKU,替代料,品类,身份,在途总数,改价,无Record ID,实时库存,组合料库存,...,断货时间,改价+断货,总库存,总品类调价记录,每日调价记录,总品类调价记录_2,合计改价记录,note,原始行号,row
0,TOP1,USBCB72928BLR-C,USBCB72928BLR,BC,老品,146.0,,3.0,70.0,1.0,...,,,71.0,,,,,,3.0,16587
1,TOP20,USPSP14045D-C,USPSP14045D,PSP,老品,6.0,,0.0,47.0,0.0,...,,,47.0,2.7-T3,,,,,4.0,15664
2,TOP20,USBCB75352ZLR-C,USBCB75352ZLR,BC,断货,34.0,,0.0,1.0,0.0,...,,,1.0,,,,,,5.0,16609
3,TOP1,USBCB08180ZLR-C,USBCB08180ZLR,BC,老品,15.0,,0.0,102.0,12.0,...,,,114.0,6.5-J2,,,,,6.0,16590
4,普通,USPSP01008D-C,USPSP01008D,PSP,老品,0.0,,0.0,19.0,0.0,...,,,19.0,,,,,,7.0,16037
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24398,普通,USPBP93656,,PBP,断货,6.0,,0.0,0.0,0.0,...,,,0.0,,,,,mrp,,
24399,普通,USCA01113-C,,CA,新品,12.0,,0.0,6.0,0.0,...,,,6.0,,,,,mrp,,33975
24400,普通,USCA01114-C,,CA,新品,13.0,,0.0,8.0,0.0,...,,,8.0,,,,,mrp,,33976
24401,TOP20,USCAK09028-C,,CA,新品,4.0,,0.0,7.0,6.0,...,,,13.0,,,,,mrp,,30228


In [56]:
if timing == 'am':
    df_sales = pd.read_excel(stcok,
                             sheet_name=2,
                             usecols='D,F')
    
    df_sales = df_sales.groupby('SKU', as_index=False).sum()

In [57]:
if timing == 'am':
    df = pd.merge(left=df, right=df_sales, how='left', left_on='SKU', right_on='SKU', suffixes=('','_sales')).fillna(0)

In [58]:
if timing == 'am':
    df_sales = pd.merge(left=df_sales, right=df_position, how='left', left_on='SKU', right_on='SKU', suffixes=('_x','')).fillna('')
    df_sales = df_sales[df_sales['row'] != ''][['购买数量', 'row']].copy()
    df_sales = df_sales.groupby('row', as_index=False).sum()
    df_sales

In [59]:
if timing == 'am':
#     df_tmp = df
    df = pd.merge(left=df, right=df_sales, how='left', left_on='row', right_on='row', suffixes=('_sales1', '_sales2')).fillna('')
    df['销量date'] = [df.loc[i, '购买数量_sales2'] if df.loc[i, '购买数量_sales2'] != '' else df.loc[i, '购买数量_sales1']  for i in range(len(df))]
    print(list(df.columns))
    
    df

['层级', 'SKU', '替代料', '品类', '身份', '在途总数', '改价', '无Record ID', '实时库存', '组合料库存', 'YKD库存', '库存date', datetime.datetime(2023, 8, 28, 0, 0, 1), datetime.datetime(2023, 8, 26, 0, 0, 1), '2023-08-28 00:00:01.1', datetime.datetime(2023, 8, 27, 0, 0, 1), '2023-08-26 00:00:01.1', '2023-08-25 00:00:01.1', datetime.datetime(2023, 8, 24, 0, 0, 1), datetime.datetime(2023, 8, 23, 0, 0, 1), datetime.datetime(2023, 8, 22, 0, 0, 1), 'weekly_sale', '改价记录', '断货时间', '改价+断货', '总库存', '总品类调价记录', '每日调价记录', '总品类调价记录_2', '合计改价记录', 'note', '原始行号', 'row', '购买数量_sales1', '购买数量_sales2', '销量date']


In [60]:
if timing == 'am':
    df = df[['层级',
             'SKU',
             '替代料',
             '品类',
             '身份',
             '在途总数',
             '改价',
             '无Record ID',
             '实时库存',
             '组合料库存',
             'YKD库存'] + list(df.columns)[11:13+1]+ ['销量date'] + list(df.columns)[14:20+1] + ['weekly_sale',
                                                                                                 '改价记录', 
                                                                                                 '断货时间', 
                                                                                                 '改价+断货',
                                                                                                 '总库存',
                                                                                                 '总品类调价记录',
                                                                                                 '每日调价记录',
                                                                                                 '总品类调价记录_2',
                                                                                                 '合计改价记录',
                                                                                                 'note',
                                                                                                 '原始行号']].copy()

print(list(df.columns))
df

['层级', 'SKU', '替代料', '品类', '身份', '在途总数', '改价', '无Record ID', '实时库存', '组合料库存', 'YKD库存', '库存date', datetime.datetime(2023, 8, 28, 0, 0, 1), datetime.datetime(2023, 8, 26, 0, 0, 1), '销量date', '2023-08-28 00:00:01.1', datetime.datetime(2023, 8, 27, 0, 0, 1), '2023-08-26 00:00:01.1', '2023-08-25 00:00:01.1', datetime.datetime(2023, 8, 24, 0, 0, 1), datetime.datetime(2023, 8, 23, 0, 0, 1), datetime.datetime(2023, 8, 22, 0, 0, 1), 'weekly_sale', '改价记录', '断货时间', '改价+断货', '总库存', '总品类调价记录', '每日调价记录', '总品类调价记录_2', '合计改价记录', 'note', '原始行号']


,层级,SKU,替代料,品类,身份,在途总数,改价,无Record ID,实时库存,组合料库存,...,改价记录,断货时间,改价+断货,总库存,总品类调价记录,每日调价记录,总品类调价记录_2,合计改价记录,note,原始行号
0,TOP1,USBCB72928BLR-C,USBCB72928BLR,BC,老品,146.0,,3.0,70.0,1.0,...,,,,71.0,,,,,,3.0
1,TOP20,USPSP14045D-C,USPSP14045D,PSP,老品,6.0,,0.0,47.0,0.0,...,,,,47.0,2.7-T3,,,,,4.0
2,TOP20,USBCB75352ZLR-C,USBCB75352ZLR,BC,断货,34.0,,0.0,1.0,0.0,...,8.24-T10,,,1.0,,,,,,5.0
3,TOP1,USBCB08180ZLR-C,USBCB08180ZLR,BC,老品,15.0,,0.0,102.0,12.0,...,,,,114.0,6.5-J2,,,,,6.0
4,普通,USPSP01008D-C,USPSP01008D,PSP,老品,0.0,,0.0,19.0,0.0,...,,,,19.0,,,,,,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24398,普通,USPBP93656,,PBP,断货,6.0,,0.0,0.0,0.0,...,,,,0.0,,,,,mrp,
24399,普通,USCA01113-C,,CA,新品,12.0,,0.0,6.0,0.0,...,,,,6.0,,,,,mrp,
24400,普通,USCA01114-C,,CA,新品,13.0,,0.0,8.0,0.0,...,,,,8.0,,,,,mrp,
24401,TOP20,USCAK09028-C,,CA,新品,4.0,,0.0,7.0,6.0,...,,,,13.0,,,,,mrp,


In [61]:
offset = 3

df['改价'] = ['=IFERROR(IF(I' + str(i+3) + '-$L' + str(i+3) + '>0,"海运上架",IF(I'+ str(i+3) + '<=0,"断货","")),"")' for i in range(len(df))]
df['weekly_sale'] = ['=SUM(O' + str(i+3) + ':V' + str(i+3) +')' for i in range(len(df))]
df['改价+断货'] = [ '=TEXTJOIN(";",1,X' + str(i+3) + ',Y' + str(i+3) + ')' for i in range(len(df))]
df['总品类调价记录_2'] = ['=TEXTJOIN(";",TRUE,AB' + str(i+3) + ':AC' + str(i+3) + ')' for i in range(len(df))]
df['合计改价记录'] = ['=TEXTJOIN(";",1,AD' + str(i+3) + ',X' + str(i+3) + ')' for i in range(len(df))]

In [62]:
print(df.columns)

Index([                   '层级',                   'SKU',
                         '替代料',                    '品类',
                          '身份',                  '在途总数',
                          '改价',            '无Record ID',
                        '实时库存',                 '组合料库存',
                       'YKD库存',                '库存date',
           2023-08-28 00:00:01,     2023-08-26 00:00:01,
                      '销量date', '2023-08-28 00:00:01.1',
           2023-08-27 00:00:01, '2023-08-26 00:00:01.1',
       '2023-08-25 00:00:01.1',     2023-08-24 00:00:01,
           2023-08-23 00:00:01,     2023-08-22 00:00:01,
                 'weekly_sale',                  '改价记录',
                        '断货时间',                 '改价+断货',
                         '总库存',               '总品类调价记录',
                      '每日调价记录',             '总品类调价记录_2',
                      '合计改价记录',                  'note',
                        '原始行号'],
      dtype='object')


In [63]:
df_tmp = df
df_tmp = pd.merge(left=df_tmp, right=df_day_category, how='left', left_on='SKU', right_on='SKU', suffixes=('','_last_record')).fillna('')
df_tmp['每日调价记录'] = df_tmp['调价记录']
print(df_tmp.columns)
# df_tmp

Index([                   '层级',                   'SKU',
                         '替代料',                    '品类',
                          '身份',                  '在途总数',
                          '改价',            '无Record ID',
                        '实时库存',                 '组合料库存',
                       'YKD库存',                '库存date',
           2023-08-28 00:00:01,     2023-08-26 00:00:01,
                      '销量date', '2023-08-28 00:00:01.1',
           2023-08-27 00:00:01, '2023-08-26 00:00:01.1',
       '2023-08-25 00:00:01.1',     2023-08-24 00:00:01,
           2023-08-23 00:00:01,     2023-08-22 00:00:01,
                 'weekly_sale',                  '改价记录',
                        '断货时间',                 '改价+断货',
                         '总库存',               '总品类调价记录',
                      '每日调价记录',             '总品类调价记录_2',
                      '合计改价记录',                  'note',
                        '原始行号',                  '调价记录'],
      dtype='object')


In [64]:
# df = df_tmp

if timing == 'am':
    df_tmp = df_tmp[['层级',
                     'SKU',
                     '替代料',
                     '品类',
                     '身份',
                     '在途总数',
                     '改价',
                     '无Record ID',
                     '实时库存',
                     '组合料库存',
                     'YKD库存'] + list(df_tmp.columns)[11:21+1] + ['weekly_sale',
                                                                     '改价记录', 
                                                                     '断货时间', 
                                                                     '改价+断货',
                                                                     '总库存',
                                                                     '总品类调价记录',
                                                                     '每日调价记录',
                                                                     '总品类调价记录_2',
                                                                     '合计改价记录',
                                                                     'note',
                                                                     '原始行号']].copy()

    print(list(df_tmp.columns))
    df_tmp
else:
    df_tmp = df_tmp[['层级',
                     'SKU',
                     '替代料',
                     '品类',
                     '身份',
                     '在途总数',
                     '改价',
                     '无Record ID',
                     '实时库存',
                     '组合料库存',
                     'YKD库存'] + list(df_tmp.columns)[11:21+1] + ['weekly_sale',
                                                                     '改价记录', 
                                                                     '断货时间', 
                                                                     '改价+断货',
                                                                     '总库存',
                                                                     '总品类调价记录',
                                                                     '每日调价记录',
                                                                     '总品类调价记录_2',
                                                                     '合计改价记录',
                                                                     'note',
                                                                     '原始行号']].copy()
# df = df_tmp
print(df_tmp.columns)
df_tmp

['层级', 'SKU', '替代料', '品类', '身份', '在途总数', '改价', '无Record ID', '实时库存', '组合料库存', 'YKD库存', '库存date', datetime.datetime(2023, 8, 28, 0, 0, 1), datetime.datetime(2023, 8, 26, 0, 0, 1), '销量date', '2023-08-28 00:00:01.1', datetime.datetime(2023, 8, 27, 0, 0, 1), '2023-08-26 00:00:01.1', '2023-08-25 00:00:01.1', datetime.datetime(2023, 8, 24, 0, 0, 1), datetime.datetime(2023, 8, 23, 0, 0, 1), datetime.datetime(2023, 8, 22, 0, 0, 1), 'weekly_sale', '改价记录', '断货时间', '改价+断货', '总库存', '总品类调价记录', '每日调价记录', '总品类调价记录_2', '合计改价记录', 'note', '原始行号']
Index([                   '层级',                   'SKU',
                         '替代料',                    '品类',
                          '身份',                  '在途总数',
                          '改价',            '无Record ID',
                        '实时库存',                 '组合料库存',
                       'YKD库存',                '库存date',
           2023-08-28 00:00:01,     2023-08-26 00:00:01,
                      '销量date', '2023-08-28 00:00:01.1',
         

,层级,SKU,替代料,品类,身份,在途总数,改价,无Record ID,实时库存,组合料库存,...,改价记录,断货时间,改价+断货,总库存,总品类调价记录,每日调价记录,总品类调价记录_2,合计改价记录,note,原始行号
0,TOP1,USBCB72928BLR-C,USBCB72928BLR,BC,老品,146.0,"=IFERROR(IF(I3-$L3>0,""海运上架"",IF(I3<=0,""断货"",""""))...",3.0,70.0,1.0,...,,,"=TEXTJOIN("";"",1,X3,Y3)",71.0,,,"=TEXTJOIN("";"",TRUE,AB3:AC3)","=TEXTJOIN("";"",1,AD3,X3)",,3.0
1,TOP20,USPSP14045D-C,USPSP14045D,PSP,老品,6.0,"=IFERROR(IF(I4-$L4>0,""海运上架"",IF(I4<=0,""断货"",""""))...",0.0,47.0,0.0,...,,,"=TEXTJOIN("";"",1,X4,Y4)",47.0,2.7-T3,,"=TEXTJOIN("";"",TRUE,AB4:AC4)","=TEXTJOIN("";"",1,AD4,X4)",,4.0
2,TOP20,USBCB75352ZLR-C,USBCB75352ZLR,BC,断货,34.0,"=IFERROR(IF(I5-$L5>0,""海运上架"",IF(I5<=0,""断货"",""""))...",0.0,1.0,0.0,...,8.24-T10,,"=TEXTJOIN("";"",1,X5,Y5)",1.0,,,"=TEXTJOIN("";"",TRUE,AB5:AC5)","=TEXTJOIN("";"",1,AD5,X5)",,5.0
3,TOP1,USBCB08180ZLR-C,USBCB08180ZLR,BC,老品,15.0,"=IFERROR(IF(I6-$L6>0,""海运上架"",IF(I6<=0,""断货"",""""))...",0.0,102.0,12.0,...,,,"=TEXTJOIN("";"",1,X6,Y6)",114.0,6.5-J2,,"=TEXTJOIN("";"",TRUE,AB6:AC6)","=TEXTJOIN("";"",1,AD6,X6)",,6.0
4,普通,USPSP01008D-C,USPSP01008D,PSP,老品,0.0,"=IFERROR(IF(I7-$L7>0,""海运上架"",IF(I7<=0,""断货"",""""))...",0.0,19.0,0.0,...,,,"=TEXTJOIN("";"",1,X7,Y7)",19.0,,,"=TEXTJOIN("";"",TRUE,AB7:AC7)","=TEXTJOIN("";"",1,AD7,X7)",,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24398,普通,USPBP93656,,PBP,断货,6.0,"=IFERROR(IF(I24401-$L24401>0,""海运上架"",IF(I24401<...",0.0,0.0,0.0,...,,,"=TEXTJOIN("";"",1,X24401,Y24401)",0.0,,,"=TEXTJOIN("";"",TRUE,AB24401:AC24401)","=TEXTJOIN("";"",1,AD24401,X24401)",mrp,
24399,普通,USCA01113-C,,CA,新品,12.0,"=IFERROR(IF(I24402-$L24402>0,""海运上架"",IF(I24402<...",0.0,6.0,0.0,...,,,"=TEXTJOIN("";"",1,X24402,Y24402)",6.0,,,"=TEXTJOIN("";"",TRUE,AB24402:AC24402)","=TEXTJOIN("";"",1,AD24402,X24402)",mrp,
24400,普通,USCA01114-C,,CA,新品,13.0,"=IFERROR(IF(I24403-$L24403>0,""海运上架"",IF(I24403<...",0.0,8.0,0.0,...,,,"=TEXTJOIN("";"",1,X24403,Y24403)",8.0,,,"=TEXTJOIN("";"",TRUE,AB24403:AC24403)","=TEXTJOIN("";"",1,AD24403,X24403)",mrp,
24401,TOP20,USCAK09028-C,,CA,新品,4.0,"=IFERROR(IF(I24404-$L24404>0,""海运上架"",IF(I24404<...",0.0,7.0,6.0,...,,,"=TEXTJOIN("";"",1,X24404,Y24404)",13.0,,,"=TEXTJOIN("";"",TRUE,AB24404:AC24404)","=TEXTJOIN("";"",1,AD24404,X24404)",mrp,


In [65]:
df_tmp.columns

Index([                   '层级',                   'SKU',
                         '替代料',                    '品类',
                          '身份',                  '在途总数',
                          '改价',            '无Record ID',
                        '实时库存',                 '组合料库存',
                       'YKD库存',                '库存date',
           2023-08-28 00:00:01,     2023-08-26 00:00:01,
                      '销量date', '2023-08-28 00:00:01.1',
           2023-08-27 00:00:01, '2023-08-26 00:00:01.1',
       '2023-08-25 00:00:01.1',     2023-08-24 00:00:01,
           2023-08-23 00:00:01,     2023-08-22 00:00:01,
                 'weekly_sale',                  '改价记录',
                        '断货时间',                 '改价+断货',
                         '总库存',               '总品类调价记录',
                      '每日调价记录',             '总品类调价记录_2',
                      '合计改价记录',                  'note',
                        '原始行号'],
      dtype='object')

In [66]:
df = df_tmp

In [67]:
from datetime import datetime

now = datetime.now()
month = now.month
day = now.day
month_day = f'{month}.{day}'

if timing == 'am':
    out = 'Bear_断货改价_Hebe_上午 ' + month_day + '.xlsx'
else:
    out = 'Bear_断货改价_Hebe_下午 ' + month_day + '.xlsx'

df.to_excel('./' + out, 
            index=False ,
            startrow=1)

# 7.改格式

In [68]:
from openpyxl import load_workbook

In [69]:
wb_last = load_workbook(last)
ws_last = wb_last.worksheets[0]
ws_last

<Worksheet "Sheet1">

In [70]:
wb_out = load_workbook('./' + out)
ws_out = wb_out.worksheets[0]

In [71]:
coef = 1.125
ws_out.column_dimensions['A'].width = coef * 5.56
ws_out.column_dimensions['B'].width = coef * 18.69
ws_out.column_dimensions['C'].width = coef * 18.69
ws_out.column_dimensions['D'].width = coef * 6.88
ws_out.column_dimensions['E'].width = coef * 6.88
ws_out.column_dimensions['F'].width = coef * 9.88
ws_out.column_dimensions['G'].width = coef * 6.88
ws_out.column_dimensions['H'].width = coef * 9.88
ws_out.column_dimensions['I'].width = coef * 9.88
ws_out.column_dimensions['J'].width = coef * 7.04
ws_out.column_dimensions['K'].width = coef * 5.5
ws_out.column_dimensions['L'].width = coef * 5.5
ws_out.column_dimensions['M'].width = coef * 5.5
ws_out.column_dimensions['N'].width = coef * 5.5
ws_out.column_dimensions['O'].width = coef * 5.5
ws_out.column_dimensions['P'].width = coef * 5.5
ws_out.column_dimensions['Q'].width = coef * 5.5
ws_out.column_dimensions['R'].width = coef * 5.5
ws_out.column_dimensions['S'].width = coef * 5.5
ws_out.column_dimensions['T'].width = coef * 5.5
ws_out.column_dimensions['U'].width = coef * 5.5
ws_out.column_dimensions['V'].width = coef * 5.5
ws_out.column_dimensions['W'].width = coef * 5.5
ws_out.column_dimensions['X'].width = coef * 10.88
ws_out.column_dimensions['Y'].width = coef * 13
ws_out.column_dimensions['Z'].width = coef * 10.88
ws_out.column_dimensions['AA'].width = coef * 10.88
ws_out.column_dimensions['AB'].width = coef * 13
ws_out.column_dimensions['AC'].width = coef * 10.88
ws_out.column_dimensions['AD'].width = coef * 10.88
ws_out.column_dimensions['AE'].width = coef * 10.88
ws_out.column_dimensions['AF'].width = coef * 8.75
ws_out.column_dimensions['AG'].width = coef * 8.75

ws_out.row_dimensions[1].height = 25
ws_out.row_dimensions[2].height = 34
for i in range(3, ws_out.max_row+1):
    ws_out.row_dimensions[i].height = 16.5

In [72]:
from openpyxl.styles import Font, PatternFill
from openpyxl.styles import *
import datetime

In [73]:
orange ='00FFCC99'
blue_dark = 'FF8DB4E2'
green = 'FF92D050'
yellow = '00FFFF00'

now = datetime.datetime.now()
month = now.month
day = now.day
month_day = f'{month}/{day}'

ws_out['A1'].value = month_day
ws_out['A1'].font = Font(bold=True)

ws_out.merge_cells('B1:E1')

ws_out['G1'].value = '任务栏'

ws_out.merge_cells('H1:K1')

ws_out['O1'].value = '近一周销量'
ws_out.merge_cells('O1:V1')

ws_out['X1'].value = '10.5-J310.18-T6(日期最新在后面）'

In [74]:
ws_out['AA2'].fill = PatternFill(patternType='solid', fgColor=yellow)

for i in ws_out['A2:J2']:
    for j in i:
        j.fill = PatternFill(patternType='solid', fgColor=blue_dark)
        
for i in ws_out['K2:N2']:
    for j in i:
        j.fill = PatternFill(patternType='solid', fgColor=green)

for i in ws_out['O2:V2']:
    for j in i:
        j.fill = PatternFill(patternType='solid', fgColor=orange)
         
for i in ws_out['W2:Z2']:
    for j in i:
        j.fill = PatternFill(patternType='solid', fgColor=blue_dark)
        
for i in ws_out['AB2:AH2']:
    for j in i:
        j.fill = PatternFill(patternType='solid', fgColor=blue_dark)
        
for i in ws_out['M2:N2']:
    for j in i:
        j.value = datetime.datetime.strptime(str(j.value)[:17] + '01', '%Y-%m-%d %H:%M:%S')
        j.number_format = 'm/d;@'
        
for i in ws_out['P2:V2']:
    for j in i:
        j.value = datetime.datetime.strptime(str(j.value)[:17] + '01', '%Y-%m-%d %H:%M:%S')
        j.number_format = 'm/d;@'
#         j.fill = PatternFill(patternType='solid', fgColor=blue_light)

In [75]:
# for i in range(3, ws_out.max_row+1):
#     row = ws_out['AH'+str(i)].value
#     if row != None:
#         if timing == 'am':
#             color_C = ws_last['D'+str(row)].font.color
#             color_N = ws_last['Z'+str(row)].font.color
#             color_M = ws_last['Y'+str(row)].font.color
#             color_L = ws_last['X'+str(row)].font.color
#             color_K = ws_last['X'+str(row)].font.color

#         else:
#             color_C = ws_last['D'+str(row)].font.color
#             color_N = ws_last['AA'+str(row)].font.color
#             color_M = ws_last['Z'+str(row)].font.color
#             color_L = ws_last['Y'+str(row)].font.color
#             color_K = ws_last['X'+str(row)].font.color

#     else:
#         color_C = None
#         color_N = None
#         color_M = None
#         color_L = None
#         color_K = None
    
#     if color_C != None:
#         color_C = color_C.rgb
#     else:
#         color_C = None  
    
#     if color_N != None:
#         color_N = color_N.rgb
#     else:
#         color_N = None
    
#     if color_M != None:
#         color_M = color_M.rgb
#     else:
#         color_M = None

#     if color_L != None:
#         color_L = color_L.rgb
#     else:
#         color_L = None
    
#     if color_K != None:
#         color_K = color_K.rgb
#     else:
#         color_K = None
        

# #     print(i,color_C,color_N, color_M, color_L, color_K)

#     if type(color_C) == str:
#         ws_out['C'+str(i)].font = Font(color=color_C)
    
#     if type(color_N) == str:
#         ws_out['N'+str(i)].font = Font(color=color_N)
    
#     if type(color_M) == str:
#         ws_out['M'+str(i)].font = Font(color=color_M)

#     if type(color_L) == str:
#         ws_out['L'+str(i)].font = Font(color=color_L)
        
#     if type(color_K) == str:
#         ws_out['K'+str(i)].font = Font(color=color_K)


In [76]:

for i in range(3, ws_out.max_row+1):
    row = ws_out['AH'+str(i)].value
    if row != None:
        if timing == 'am':
            color_C = ws_last['C'+str(row)].font.color
            color_N = ws_last['M'+str(row)].font.color
            color_M = ws_last['L'+str(row)].font.color
            color_L = ws_last['K'+str(row)].font.color
            color_K = ws_last['K'+str(row)].font.color

        else:
            color_C = ws_last['C'+str(row)].font.color
            color_N = ws_last['N'+str(row)].font.color
            color_M = ws_last['M'+str(row)].font.color
            color_L = ws_last['L'+str(row)].font.color
            color_K = ws_last['K'+str(row)].font.color
            
    else:
        color_C = None
        color_N = None
        color_M = None
        color_L = None
        color_K = None
    
    if color_C != None:
        color_C = color_C.rgb
    else:
        color_C = None  
    
    if color_N != None:
        color_N = color_N.rgb
    else:
        color_N = None
    
    if color_M != None:
        color_M = color_M.rgb
    else:
        color_M = None

    if color_L != None:
        color_L = color_L.rgb
    else:
        color_L = None
    
    if color_K != None:
        color_K = color_K.rgb
    else:
        color_K = None
        

#     print(i,color_C,color_N, color_M, color_L, color_K)

    if type(color_C) == str:
        ws_out['C'+str(i)].font = Font(color=color_C)
    
    if type(color_N) == str:
        ws_out['N'+str(i)].font = Font(color=color_N)
    
    if type(color_M) == str:
        ws_out['M'+str(i)].font = Font(color=color_M)

    if type(color_L) == str:
        ws_out['L'+str(i)].font = Font(color=color_L)
        
    if type(color_K) == str:
        ws_out['K'+str(i)].font = Font(color=color_K)


In [77]:
from datetime import datetime
import time

In [78]:
wb_out.save('./' + out.replace('Bear_', 'color_Bear_'))
time.sleep(3)
wb_out.close()
print(datetime.now())

2023-08-30 10:15:22.740545
